# ***Automated AARP-DS-TU model training***

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

#Import XGBoost Classifier Model
from sklearn.metrics import classification_report, confusion_matrix, roc_curve , roc_auc_score
from sklearn import metrics
from sklearn.metrics import accuracy_score , precision_score , recall_score , f1_score
from platform import python_version
print(python_version())
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV , RandomizedSearchCV


from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neural_network import MLPClassifier
import lightgbm as lgm
from lightgbm import LGBMClassifier , LGBMRanker
from collections import Counter

3.8.5


In [2]:
aarp = pd.read_csv('/mnt/data/prod/hig/projects/models/model_build_2023/gm_non_gm_append_aarp.txt',
                  sep='|')
aarp.shape

(1763366, 333)

In [3]:
aarp1=pd.read_csv('/mnt/data/prod/hig/projects/models/model_build_2023/gm_non_gm_append_aarp_mbr_variables.txt',
                  sep='|')
aarp1.shape

(1763336, 14)

In [4]:
aarp['AARP_PRIMARY_MEM_IND'].value_counts(dropna=False)

Y      1024947
N       476473
NaN     261946
Name: AARP_PRIMARY_MEM_IND, dtype: int64

In [4]:
ds = pd.read_csv('/mnt/data/prod/hig/projects/models/model_build_2023/gm_non_gm_append_ds_v1.csv')
ds.shape

(1763336, 526)

In [2]:
tu = pd.read_csv('/mnt/data/prod/hig/projects/models/model_build_2023/gm_non_gm_append_tu.csv')
tu.shape

(1763336, 413)

In [6]:
aarp.columns = map(str.upper , aarp.columns)
ds.columns = map(str.upper , ds.columns)
tu.columns = map(str.upper , tu.columns)

In [7]:
ds.rename(columns={'MATCH_IND':'DS_MATCH_IND'},inplace=True)
tu.rename(columns={'MATCH_IND':'TU_MATCH_IND'},inplace=True)

In [8]:
aarp['SP_EMAILABLE_IND'] = aarp['SP_EMAILABLE_IND'].map({'Y':1,'N':0})
aarp['AARP_SECONDARY_MEMBERSHIP_IND'] = aarp['AARP_SECONDARY_MEMBERSHIP_IND'].map({'Y':1,'N':0})

ds['DS_MATCH_IND'] = ds['DS_MATCH_IND'].map({'Y':1,'N':0})

tu['TU_MATCH_IND'] = tu['TU_MATCH_IND'].map({'Y':1,'N':0})

In [9]:
adlst = ['TU_MATCH_IND',
# 'DS_MATCH_IND',
'SP_EMAILABLE_IND',
'AARP_SECONDARY_MEMBERSHIP_IND'
        ]
len(adlst)

3

In [10]:
prod = ["AARP_ADDR_MOVE_DT_d",
"AARP_AGE", # remove birth wale
"AARP_IBX_NETWORTH_PR",
"AARP_IBX_TOT_ONLINE_DOLLARS",
"AARP_IBX_TOT_ONLINE_PURCHASES",
"AARP_IBX_TRND4TELCOM_OPTCALL_SVC",
"AARP_MEM_ORIGIN_DT_d",
"AARP_MEM_STATUS",
"AARP_MEM_TERM_MO_QTY",
"AARP_NUM_CURPRT_OVERALL_AGG_HH",
"AARP_NUM_HISTPRT_FINAN_AGG_HH",
"AARP_NUM_HISTPRT_LFSTYL_AGG_HH",
"DS_MD002",
"DS_MD006",
# "DS_MS527",
# "DS_MV006",
"DS_XB831",
"TU_ACCTS_MOS_OLD_OPEN",
"TU_AVG_MOS_ACCTS_ON_FILE",
"TU_BCARD_ACCTS_MOS_RCNT_OPEN",
"TU_BCARD_ACCTS_OPEN_0TO24M",
# "TU_CHAZ04", # remove this and add hig score
# "TU_CHAZ09", # remove this and add hig score
"TU_INFERRED_AGE", # remove birth wale
"TU_LENGTH_OF_RESIDENCE",
"TU_NUM_INS_INQ_0TO12M",
"TU_NUM_INS_INQ_0TO18M",
# "TU_NUM_INS_INQ_0TO21M",
"TU_NUM_INS_INQ_0TO24M",
"TU_PBCARD_ACCTS_AVG_BAL",
"TU_PUBREC_TRDLNE_DEROG_GT100DOL",
"HH_NBR_MOS_SINCE_LAST_MAIL",
"IND_NBR_MOS_SINCE_LAST_MAIL",

]
len(prod)

28

# reading the data

In [2]:
df = pd.read_csv('/mnt/data/prod/hig/projects/models/model_build_2023/ModelCode/ResponseModelMay23/NewResponseData/ModelData_Response_Train_01Jun23.csv')
df.shape

(220658, 861)

In [3]:
df['SEQ_NO'] = df['SEQ_NO'].astype('int')

In [4]:
dftest = pd.read_pickle('dataset/kunal_Response_Test_imputed1.pkl')
dftest.shape

(94567, 861)

In [5]:
dftest['SEQ_NO'] = dftest['SEQ_NO'].astype('int')

In [6]:
dftest.isna().sum().reset_index()[dftest.isna().sum().reset_index()[0]>0]

index      0
848    RESPONSE_DATE  75544
850  CONVERSION_DATE  90473
859            state      4
860         zip_code      4

In [7]:
df['RESPONSE_CNT'].value_counts()

0.0    176636
1.0     44022
Name: RESPONSE_CNT, dtype: int64

In [8]:
dftest['RESPONSE_CNT'].value_counts()

0.0    75544
1.0    19023
Name: RESPONSE_CNT, dtype: int64

# get sp_emailable , aarp_sec_mem_ind , tu_match_ind

##  train

In [18]:
df =pd.merge(df,aarp[['SEQ_NO','SP_EMAILABLE_IND','AARP_SECONDARY_MEMBERSHIP_IND',
                      'STATE',
                      'AARP_AGE'
                     ]],on='SEQ_NO',how='left')
df.shape

(220658, 865)

In [19]:
df['STATE'] = df['STATE'].astype('str')

In [20]:
df =pd.merge(df,ds[['SEQ_NO','DS_MATCH_IND']],on='SEQ_NO',how='left')
df.shape

(220658, 866)

In [21]:
df =pd.merge(df,tu[['SEQ_NO','TU_MATCH_IND']],on='SEQ_NO',how='left')
df.shape

(220658, 867)

##  test

In [22]:
dftest =pd.merge(dftest,aarp[['SEQ_NO','SP_EMAILABLE_IND','AARP_SECONDARY_MEMBERSHIP_IND',
                      'STATE','AARP_AGE'
                             ]],on='SEQ_NO',how='left')
dftest.shape

(94567, 865)

In [23]:
dftest['STATE'] = dftest['STATE'].astype('str')

In [24]:
dftest =pd.merge(dftest,ds[['SEQ_NO','DS_MATCH_IND']],on='SEQ_NO',how='left')
dftest.shape

(94567, 866)

In [25]:
dftest =pd.merge(dftest,tu[['SEQ_NO','TU_MATCH_IND']],on='SEQ_NO',how='left')
dftest.shape

(94567, 867)

# reading the missing ind data

In [26]:
train = pd.read_csv('dataset/train_seq_no.csv')
test = pd.read_csv('dataset/test_seq_no.csv')
miss = pd.read_csv('/mnt/data/prod/hig/projects/models/model_build_2023/ModelCode/ResponseModelMay23/NewResponseData/ModelData_MissingFlag_01Jun23.csv')

miss['SEQ_NO'] = miss['SEQ_NO'].astype('int')

dfmiss = miss[miss['SEQ_NO'].isin(train['SEQ_NO'])]

dftestmiss = miss[miss['SEQ_NO'].isin(test['SEQ_NO'])]
print(dfmiss.shape)
print(dftestmiss.shape)

(220658, 849)
(94567, 849)


In [27]:
dfmiss['RESPONSE_CNT'].value_counts()

0.0    176636
1.0     44022
Name: RESPONSE_CNT, dtype: int64

In [28]:
dftest['RESPONSE_CNT'].value_counts()

0.0    75544
1.0    19023
Name: RESPONSE_CNT, dtype: int64

# missing ind list

In [29]:
mls = [
#     'DS_AU02E_st',
 'TU_AUTO_XDT_d_st', # final3
#  'TU_Q26_SCORE_st',
#  'AARP_IBX_HM_ROOM_COUNT_st',
#  'TU_PBCARD_ACCTS_MOS_OLD_OPEN_st',
#  'TU_INQ_MOS_RCNT_st',
#  'TU_REV_ACCTS_AVG_BALANCE_st',
#  'DS_XM907_st',
#  'TU_PBCARD_ACCTS_AVG_BAL_st',
#  'TU_MTG_ACCT_MOS_RCNT_OPEN_st',
#  'AARP_ADDR_MOVE_DT_d_st',
#  'AARP_IBX_HLTH_BEAUTY_st',
 'TU_UNKNOWN_XDT_d_st', # final3
 'AARP_IBX_TOT_ONLINE_DOLLARS_st', # final 3
#  'DS_VW132_st',
#  'AARP_IBX_HM_OWNER_RENTER_PR_st'
      ]
len(mls)

3

In [30]:
mls.append('SEQ_NO')
len(mls)

4

In [31]:
dfmiss1 = dfmiss[mls]
dfmiss1.shape

(220658, 4)

In [32]:
dftestmiss1 = dftestmiss[mls]
dftestmiss1.shape

(94567, 4)

In [33]:
df = pd.merge(df,dfmiss1,on='SEQ_NO',how='left')
df.shape

(220658, 870)

In [34]:
dftest = pd.merge(dftest,dftestmiss1,on='SEQ_NO',how='left')
dftest.shape

(94567, 870)

In [35]:
mls.remove('SEQ_NO')
len(mls)

3

# recfreq

In [36]:
rfcols = [
    'IND_NBR_CONTACTS_3M', 'IND_NBR_CONTACTS_6M', 'IND_NBR_CONTACTS_9M',
       'IND_NBR_CONTACTS_12M', 'IND_NBR_MOS_SINCE_LAST_MAIL',
       'HH_NBR_CONTACTS_3M', 'HH_NBR_CONTACTS_6M', 'HH_NBR_CONTACTS_9M',
       'HH_NBR_CONTACTS_12M', 'HH_NBR_MOS_SINCE_LAST_MAIL']
len(rfcols)

10

In [37]:
rf = pd.read_csv('/mnt/data/prod/hig/projects/models/model_build_2023/gm_non_gm_append_recfreq.csv')
rf.shape

(1763336, 20)

In [38]:
rf = rf[['SEQ_NO',
       'IND_NBR_CONTACTS_3M', 'IND_NBR_CONTACTS_6M', 'IND_NBR_CONTACTS_9M',
       'IND_NBR_CONTACTS_12M', 'IND_NBR_MOS_SINCE_LAST_MAIL',
       'HH_NBR_CONTACTS_3M', 'HH_NBR_CONTACTS_6M', 'HH_NBR_CONTACTS_9M',
       'HH_NBR_CONTACTS_12M', 'HH_NBR_MOS_SINCE_LAST_MAIL']]
rf.shape

(1763336, 11)

# outlier treatment for recfreq vars

In [39]:
outlier_trt_num = []
for i in rf.columns[~rf.columns.isin(['SEQ_NO'])]:
    rf[i] = np.where(rf[i]>np.percentile(rf[i].dropna(),q=99),np.percentile(rf[i].dropna(),q=99),rf[i])
    rf[i] = np.where(rf[i]<np.percentile(rf[i].dropna(),q=1),np.percentile(rf[i].dropna(),q=1),rf[i])
    nm= i , (rf[i].quantile(0.99)), (rf[i].quantile(0.01))
    outlier_trt_num.append(nm)

mvi_trt_num = []
for i in rf.columns[~rf.columns.isin(['SEQ_NO'])]:
    rf[i] = rf[i].fillna(rf[i].median())
    mvi_num = i , (rf[i].median())
    mvi_trt_num.append(mvi_num)
print(len(outlier_trt_num))
print(len(mvi_trt_num))

10
10


In [40]:
df = pd.merge(df,rf,on='SEQ_NO',how='left')
df.shape

(220658, 880)

In [41]:
dftest = pd.merge(dftest,rf,on='SEQ_NO',how='left')
dftest.shape

(94567, 880)

# RRI CRI

## For RRI

# train

In [42]:
state_dt=pd.read_csv("/mnt/data/prod/hig/users/himanshi/GM/rri_hig_gm_2022.csv")
state_dt.shape

(54, 9)

In [43]:
state_dt.columns = state_dt.columns.str[-4:]

In [44]:
state_dt.columns

Index(['tate', '1121', '1221', '0122', '0222', '0322', '0422', '0522', '0622'], dtype='object')

In [45]:
state_dt.rename(columns={'tate':'STATE'},inplace=True)

In [46]:
from pandas.tseries.offsets import DateOffset

In [47]:
df['DROP_DATE'] = df['DROP_DATE'].astype('int')

In [48]:
df['DROP_DATE_my'] = pd.to_datetime(df['DROP_DATE'].astype('str'))

In [49]:
df['DROP_DATE_my'] = pd.to_datetime(df['DROP_DATE_my'].astype('str')) -DateOffset(months=2)

In [50]:
df['DROP_DATE_my'] = df['DROP_DATE_my'].astype('str').str[5:7] + df['DROP_DATE_my'].astype('str').str[2:4] 

In [51]:
rri = []
for i , j in zip(df['DROP_DATE_my'],df['STATE']):
    if j == 'nan':
        rri.append(1)
    else:
        rri.append(float(state_dt[i][state_dt['STATE'] == j]))
len(rri)

220658

In [52]:
df['RRI'] = rri
df.shape

(220658, 882)

## For RRI

# test

In [53]:
state_dt=pd.read_csv("/mnt/data/prod/hig/users/himanshi/GM/rri_hig_gm_2022.csv")
state_dt.shape

(54, 9)

In [54]:
state_dt.columns = state_dt.columns.str[-4:]

In [55]:
state_dt.columns

Index(['tate', '1121', '1221', '0122', '0222', '0322', '0422', '0522', '0622'], dtype='object')

In [56]:
state_dt.rename(columns={'tate':'STATE'},inplace=True)

In [57]:
from pandas.tseries.offsets import DateOffset

In [58]:
dftest['DROP_DATE'] = dftest['DROP_DATE'].astype('int')

In [59]:
dftest['DROP_DATE_my'] = pd.to_datetime(dftest['DROP_DATE'].astype('str'))

In [60]:
dftest['DROP_DATE_my']

0       2022-06-20
1       2022-01-24
2       2022-08-22
3       2022-03-21
4       2022-01-24
           ...    
94562   2022-03-21
94563   2022-02-21
94564   2022-01-24
94565   2022-01-24
94566   2022-02-21
Name: DROP_DATE_my, Length: 94567, dtype: datetime64[ns]

In [61]:
dftest['DROP_DATE_my'] = pd.to_datetime(dftest['DROP_DATE_my'].astype('str')) -DateOffset(months=2)

In [62]:
dftest['DROP_DATE_my']

0       2022-04-20
1       2021-11-24
2       2022-06-22
3       2022-01-21
4       2021-11-24
           ...    
94562   2022-01-21
94563   2021-12-21
94564   2021-11-24
94565   2021-11-24
94566   2021-12-21
Name: DROP_DATE_my, Length: 94567, dtype: datetime64[ns]

In [63]:
dftest['DROP_DATE_my'] = dftest['DROP_DATE_my'].astype('str').str[5:7] + dftest['DROP_DATE_my'].astype('str').str[2:4] 

In [64]:
rri = []
for i , j in zip(dftest['DROP_DATE_my'],dftest['STATE']):
    if j == 'nan':
        rri.append(1)
    else:
        rri.append(float(state_dt[i][state_dt['STATE'] == j]))
len(rri)

94567

In [65]:
dftest['RRI'] = rri
dftest.shape

(94567, 882)

## For CRI

# train

In [66]:
state_dt=pd.read_csv("/mnt/data/prod/hig/users/himanshi/GM/cri_state_6_2022.csv")
state_dt.shape

(51, 91)

In [67]:
state_dt.columns = state_dt.columns.str[-4:]

In [68]:
state_dt.columns

Index(['TATE', '0415', '0316', '0416', '0315', '0515', '0615', '0715', '0815',
       '0915', '0516', '0616', '0716', '0816', '0916', '1016', '1116', '1216',
       '0117', '0217', '0317', '0417', '0116', '0216', '1015', '1115', '1215',
       '0517', '0617', '0717', '0817', '0917', '1017', '1117', '1217', '0118',
       '0218', '0318', '0418', '0518', '0618', '0718', '0818', '0918', '1018',
       '1118', '1218', '0119', '0219', '0319', '0419', '0519', '0619', '0719',
       '0819', '0919', '1019', '1119', '1219', '0120', '0220', '0320', '0420',
       '0520', '0620', '0720', '0820', '0920', '1020', '1120', '1220', '0121',
       '0221', '0321', '0421', '0521', '0621', '0721', '0821', '0921', '1021',
       '1121', '1221', '0122', '0222', '0322', '0422', '0522', '0622', '0722',
       '0822'],
      dtype='object')

In [69]:
state_dt.rename(columns={'TATE':'STATE'},inplace=True)

In [70]:
rri = []
for i , j in zip(df['DROP_DATE_my'],df['STATE']):
    if j == 'nan':
        rri.append(1)
    else:
        rri.append(float(state_dt[i][state_dt['STATE'] == j]))
len(rri)

220658

In [71]:
df['CRI'] = rri
df.shape

(220658, 883)

## For CRI

# test

In [72]:
state_dt=pd.read_csv("/mnt/data/prod/hig/users/himanshi/GM/cri_state_6_2022.csv")
state_dt.shape

(51, 91)

In [73]:
state_dt.columns = state_dt.columns.str[-4:]

In [74]:
state_dt.columns

Index(['TATE', '0415', '0316', '0416', '0315', '0515', '0615', '0715', '0815',
       '0915', '0516', '0616', '0716', '0816', '0916', '1016', '1116', '1216',
       '0117', '0217', '0317', '0417', '0116', '0216', '1015', '1115', '1215',
       '0517', '0617', '0717', '0817', '0917', '1017', '1117', '1217', '0118',
       '0218', '0318', '0418', '0518', '0618', '0718', '0818', '0918', '1018',
       '1118', '1218', '0119', '0219', '0319', '0419', '0519', '0619', '0719',
       '0819', '0919', '1019', '1119', '1219', '0120', '0220', '0320', '0420',
       '0520', '0620', '0720', '0820', '0920', '1020', '1120', '1220', '0121',
       '0221', '0321', '0421', '0521', '0621', '0721', '0821', '0921', '1021',
       '1121', '1221', '0122', '0222', '0322', '0422', '0522', '0622', '0722',
       '0822'],
      dtype='object')

In [75]:
state_dt.rename(columns={'TATE':'STATE'},inplace=True)

In [76]:
rri = []
for i , j in zip(dftest['DROP_DATE_my'],dftest['STATE']):
    if j == 'nan':
        rri.append(1)
    else:
        rri.append(float(state_dt[i][state_dt['STATE'] == j]))
len(rri)

94567

In [77]:
dftest['CRI'] = rri
dftest.shape

(94567, 883)

# pass the list of varibales

In [78]:
ls = ['AARP_MEM_ORIGIN_DT_d',
#  'DS_MD001',
 'AARP_NUM_HISTPRT_FINAN_AGG_HH',
 'TU_BIRTH_DT_d', # remove brith wala
 'AARP_NUM_HISTPRT_OVERALL_AGG_HH',
 'DS_MS545',
 'DS_MS410',
 'DS_MS544',
 'AARP_IBX_HM_ROOF_TYPE',
 'AARP_NUM_CURPRT_OVERALL_AGG_HH',
 'AARP_MEM_TERM_MO_QTY',
 'DS_DF004',
 'AARP_ADDR_MOVE_DT_d',
 'AARP_IBX_NETWORTH_PR',
 'TU_AUTO_XDT_d',
 'AARP_PRIMARY_BIRTH_DT_d',
 'DS_AU02D',
 'AARP_IBX_TOT_ONLINE_DOLLARS',
 'DS_MD006',
 'DS_MS543',
 'TU_NUM_INS_INQ_0TO21M',
 'DS_MS496',
 'AARP_IBX_TRND4TELCOM_OPTCALL_SVC',
 'AARP_IBX_PERSONIC_CLUSTER',
 'DS_MS530',
 'TU_AVG_MOS_ACCTS_ON_FILE',
#  'DS_MT014',
 'AARP_LIFESTAGE_SEGMENT',
 'DS_MV028',
 'TU_UNKNOWN_XDT_d',
#  'DS_AU02E',
 'AARP_PRIMARY_MEM_IND',
 'DS_XB977',
 'AARP_IBX_HM_OWNER_RENTER_PR',
 'AARP_PAID_THROUGH_DT_d',
 'AARP_NUM_CTCT_OVERALL_12M_AGG_HH',
 'DS_MD002',
 'AARP_NMAS_44_CELL',
#  'AARP_IBX_TOT_ONLINE_PURCHASES',
 'TU_NUM_INS_INQ_0TO24M',
      'RRI',
      'CRI',
      'TU_HIGSCORE_SHFD01',
                        # additional vars
      'AARP_INSERT_DATE_d',
'AARP_IBX_BASE_RECORD_VERIFY_DT_d',
'AARP_MEM_X_RENEW',
'DS_MD051'
    ]
len(ls)

43

In [79]:
ls1 = [
    'DS_DS981',
 'AARP_IBX_MAIL_ORDER_BUYER',
 'AARP_IBX_HLTH_BEAUTY',
 'DS_AU014',
 'DS_MD014',
 'AARP_NMAS_35_RENEWAL',
 'DS_MV042',
 'TU_PBCARD_ACCTS_MOS_OLD_OPEN',
 'AARP_NMAS_15_LIV_COM',
 'AARP_IBX_OUTDOORS_DIMENSION',
 'DS_MD004',
 'TU_ACCTS_OPEN_0TO12M',
 'AARP_NMAS_41_MAIL',
 'DS_XB831',
 'DS_MS052',
 'AARP_NMAS_5_LIFE_INS',
 'AARP_IBX_HM_LOT_SQFT_RANGE',
 'DS_MV039',
 'AARP_IBX_HM_ROOF_TYPE',
 'DS_MT033',
 'TU_ACCTS_DISPUTED',
 'DS_MS537',
 'DS_DS951',
 'AARP_IBX_EXERCISE_HLTH_GRP_PR',
 'TU_AUTO_XDT_d',
 'AARP_NUM_CTCT_FINAN_12M_AGG_HH',
 'AARP_IBX_TOT_ONLINE_DOLLARS',
 'DS_MT010',
 'TU_BREV_ACCTS_MOS_OLD_OPEN',
 'DS_MT039',
 'TU_ACCTS_MOS_OLD_OPEN',
 'AARP_IBX_HM_SQFT_RANGE',
 'AARP_NMAS_6_FIN_SEC',
 'AARP_NMAS_31_DRVS',
 'AARP_NMAS_14_COM_INV',
 'AARP_NMAS_17_CHARITY',
 'DS_AU029',
 'AARP_NMAS_40_LTC',
 'AARP_IBX_LENGTH_OF_RESIDENCE_PR',
 'TU_REV_ACCTS_MOS_OLD_OPEN',
 'AARP_IBX_EDUCATION_INPUT_INDV_PR',
 'AARP_IBX_TRAVEL_CRUISE_VACA_PR',
 'TU_MOS_ON_FILE',
 'DS_MS496',
 'TU_PBCARD_ACCTS_MOS_RCNT_OPEN',
 'AARP_IBX_PERSONIC_CLUSTER',
 'AARP_IBX_HM_PURCHSE_DT_MO_PR',
 'DS_MT022',
 'AARP_IBX_MAIL_ORDER_RESPONDER_PR',
 'TU_DSCARD_ACCTS_VERIF_0TO3M',
 'DS_MS061',
 'TU_UNKNOWN_XDT_d',
 'DS_MT038',
 'DS_XB977',
 'TU_BCARD_ACCTS_OPEN_0TO24M',
 'AARP_PRIMARY_MEM_IND',
 'AARP_NUM_HISTPRT_TECH_AGG_HH',
 'AARP_PRIMARY_AGE',
 'DS_DS982',
 'DS_MT037',
 'TU_R74_SCORE',
 'AARP_IBX_HM_LOT_BUILT_RANGE',
 'AARP_NMAS_46_TECH',
 'AARP_IBX_TOT_ONLINE_PURCHASES',
 'AARP_IBX_HM_MARKET_VALUE_DECILES',
 'DS_ME001',
 'AARP_MEM_ORIGIN_DT_d',
 'TU_BCARD_ACCTS_OPEN_0TO18M',
 'DS_MS527',
 'DS_MD001',
 'TU_BCARD_ACCTS_AVG_BAL_ALL',
 'AARP_NMAS_26_WEBUSE',
 'AARP_IBX_PROPERTY_TYPE_PREM',
 'AARP_NMAS_3_REN_HLTH',
 'TU_NUM_INS_INQ_0TO15M',
 'DS_XB876',
 'TU_ACCTS_OPEN_0TO6M',
 'TU_BIRTH_DT_d',
 'TU_RET_ACCTS_MOS_OLD_OPEN',
 'AARP_NUM_HISTPRT_OVERALL_AGG_HH',
 'DS_MS550',
 'DS_MS173',
 'AARP_IBX_HM_OWNER',
 'AARP_NUM_CURPRT_OVERALL_AGG_HH',
 'AARP_NUM_CURPRT_LFSTYL_AGG_HH',
 'DS_DS972',
 'DS_MV049',
 'DS_DF004',
 'DS_MS172',
 'DS_MD030',
 'AARP_PRIMARY_BIRTH_DT_d',
 'AARP_MEM_STATUS',
 'DS_DS927',
 'AARP_IBX_RET_PURCH_ACT_DT_CATEGORY3',
 'DS_DS983',
 'TU_ACCTS_MOS_RCNT_OPEN',
 'DS_AU02D',
 'TU_FIN_ACCTS_VERIF_0TO12M',
 'DS_MS547',
 'TU_FIN_ACCTS_MOS_RCNT_OPEN',
 'DS_DF003',
 'DS_AU002',
 'DS_MV011',
 'DS_MS543',
 'DS_MS539',
 'TU_REV_ACCTS_AVG_BALANCE',
 'DS_MV032',
 'DS_MS548',
 'DS_MD018',
 'AARP_NMAS_50_FINANCE',
 'DS_DS971',
 'DS_MS530',
 'TU_BCARD_ACCTS_MOS_RCNT_OPEN',
 'TU_AVG_MOS_ACCTS_ON_FILE',
 'TU_BINSTALL_ACCTS_SAT',
 'DS_MS138',
 'AARP_SECONDARY_GENDER',
 'AARP_NUM_CTCT_TECH_12M_AGG_HH',
 'TU_INFERRED_AGE',
 'DS_AU02E',
 'DS_ME002',
 'DS_MT006',
 'DS_MT043',
 'TU_PUBREC_TRDLNE_DEROG_GT100DOL',
 'AARP_NMAS_23_MOTORNG',
 'TU_NUM_INS_INQ_0TO6M',
 'AARP_IBX_HM_OWNER_RENTER_PR',
 'DS_MS957',
 'TU_RET_ACCTS_OPEN_0TO18M',
 'DS_AU017',
 'AARP_IBX_HM_PURCHASE_DT_PR_d',
 'AARP_NMAS_1_HLTH_DCL',
 'DS_MT005',
 'AARP_NUM_CTCT_OVERALL_12M_AGG_HH',
 'DS_MS529',
 'AARP_IBX_EDUCATION_INDV2_PR',
 'DS_MS538',
 'TU_MULTI_DWELLING_UNIT',
 'DS_MV038',
 'TU_Q26_SCORE',
 'AARP_NMAS_45_WEB',
 'AARP_NMAS_36_UTILITY',
 'AARP_IBX_HM_MARKET_VALUE_PR',
 'DS_AU013',
 'TU_RET_ACCTS_MOS_RCNT_OPEN',
 'TU_PO_BOX_IND',
 'DS_XB058',
 'AARP_IBX_TRND4TELCOM_CELLULAR',
 'DS_DS969',
 'AARP_NMAS_43_PHONE',
 'TU_BAU2PTOT',
 'AARP_IBX_CREDIT_CARD_USER',
 'AARP_NMAS_13_CAR_INS',
 'DS_MS540',
 'TU_NUM_INS_INQ_0TO18M',
 'TU_LENGTH_OF_RESIDENCE',
 'DS_DS956',
 'DS_DS928',
 'DS_MS181',
 'DS_MS410',
 'DS_MD007',
 'DS_MS544',
 'TU_PERS_FIN_ACCTS_OPEN_18M',
 'DS_MT001',
 'DS_MS518',
 'AARP_IBX_INCOME_EST_NARROWRNG_PR',
 'DS_MV045',
 'DS_DS985',
 'DS_MV036',
 'AARP_IBX_TRAVEL_GRP_PR',
 'AARP_ADDR_MOVE_DT_d',
 'AARP_NUM_CTCT_TECH_3M_AGG_HH',
 'TU_NUM_INS_INQ_0TO12M',
 'TU_ACCTS_TOT_HC_CRDLMT',
 'DS_MV021',
 'DS_MT049',
 'AARP_IBX_NUM_OF_LINES_OF_CREDIT',
 'DS_MV043',
 'DS_XB952',
 'DS_MV008',
 'TU_NUM_INS_INQ_0TO21M',
 'TU_ACCTS_W_HIGH_MOP',
 'DS_VW107',
 'DS_XM907',
 'AARP_IBX_TRND4TELCOM_OPTCALL_SVC',
 'DS_MV037',
 'DS_MT014',
 'DS_MV047',
 'AARP_IBX_RET_PURCH_ACT_DT_CATEGORY2',
 'DS_AU003',
 'AARP_NUM_CTCT_AARP_12M_AGG_HH',
 'TU_ACCTS_EVER_120PLUSDAYPASTDUE',
 'TU_DSCARD_ACCTS_MOS_RCNT_OPEN',
 'AARP_LIFESTAGE_SEGMENT',
 'AARP_NUM_CTCT_AARP_3M_AGG_HH',
 'DS_AU020',
 'AARP_IBX_OWNER_TYPE_DETAIL',
 'AARP_IBX_INFERRED_HH_RANK',
 'DS_MV028',
 'AARP_IBX_PC_OWNER_PREM',
 'DS_MT021',
 'DS_MT041',
 'DS_MS293',
 'AARP_NUM_CTCT_OVERALL_3M_AGG_HH',
 'TU_BCARD_ACCTS_OPEN_0TO12M',
 'DS_AU006',
 'DS_DS987',
 'DS_DS977',
 'TU_NUM_INS_INQ_0TO3M',
 'DS_DF009',
 'TU_OTHER_FIN_ACCTS_CURR_BAL_GT0',
 'DS_MV002',
 'AARP_MARITAL_STATUS',
 'TU_NUM_INS_INQ_0TO2M',
 'AARP_PAID_THROUGH_DT_d',
 'DS_MD002',
 'DS_DS952',
 'AARP_NMAS_20_TRAVEL',
 'AARP_IBX_OVERALL_MATCH_IDT',
 'AARP_IBX_TRAVEL_US_PR',
 'AARP_NMAS_44_CELL',
 'DS_MT045',
 'DS_MV007',
 'DS_AU02B',
 'AARP_NMAS_22_CHILDRN',
 'DS_MS947',
 'TU_STRATUM_CLUSTER_CD_NEW',
 'AARP_IBX_HM_PURCHSE_DT_YR_PR',
 'DS_MV010',
 'DS_DS974',
 'DS_AU005',
 'DS_MS542',
 'AARP_NUM_HISTPRT_FINAN_AGG_HH',
 'DS_MV012',
 'DS_MD003',
 'DS_MS134',
 'TU_INQS',
 'AARP_NMAS_11_RETIRE',
 'DS_VW105',
 'DS_MV041',
 'AARP_IBX_TRND4TELCOM_INTERNET',
 'AARP_IBX_HM_ASSESSED_VALUE_RANGE',
 'AARP_IBX_HH_INCOME',
 'DS_MS545',
 'AARP_IBX_ELECTRONICS_CO_GRP_PR',
 'AARP_IBX_CHILDREN_PRESENCE_OF_HH',
 'DS_VW132',
 'TU_BANK_FIN_ACCTS_SAT',
 'DS_MS551',
 'DS_DS953',
 'DS_MS112',
 'DS_MS113',
 'DS_MS535',
 'AARP_MEM_TERM_MO_QTY',
 'DS_MS233',
 'AARP_IBX_NETWORTH_PR',
 'AARP_IBX_EDUCATION_INDV1',
 'DS_MD019',
 'AARP_NMAS_4_PARENT',
 'DS_DS978',
 'AARP_NMAS_37_REN_OT',
 'DS_MD006',
 'TU_ACCTS_OPEN_0TO24M',
 'AARP_NMAS_29_LIFE',
 'DS_MS546',
 'AARP_IBX_DWELLING_TYPE',
 'AARP_NMAS_34_BIZNES',
 'TU_REV_ACCTS_TOT_HC_CRDLMT',
 'AARP_IBX_WORKING_WOMAN_PR',
 'DS_MV022',
 'DS_DS979',
 'AARP_IBX_OCCUPATN_INDV1_PR',
 'DS_MS520',
 'AARP_NUM_HISTPRT_LFSTYL_AGG_HH',
 'DS_AU018',
 'AARP_NMAS_25_ATTITDE',
 'TU_HIGSCORE_SHFD01',
 'DS_MV006',
 'TU_BCARD_ACCTS',
 'AARP_NMAS_10_CREDIT',
 'AARP_NMAS_39_SS',
 'TU_NUM_INS_INQ_0TO9M',
 'DS_MD024',
 'AARP_IBX_HM_PROPERTY_TYPE_DETAIL',
 'DS_MS133',
 'DS_VW513',
 'AARP_MEM_X_RENEW',
 'DS_MS528',
 'DS_MS536',
 'DS_MD021',
 'TU_BAU2ATOT',
 'AARP_IBX_TRND4TEL_TOP20LNGDIST',
 'DS_VW249',
 'TU_NUM_INS_INQ_0TO24M',
 'AARP_IBX_TRND4TELCOM_INTLNGDIST',
 'DS_XB951',
 'TU_NUM_INS_INQ_0TO4M',
 'TU_BANK_FIN_PERS_REV_ACCTS'

]
len(ls1)

298

In [80]:
ls = ls+ls1
len(ls)

341

In [81]:
ls = list(set(ls))
len(ls)

303

In [82]:
rf_lst = ['IND_NBR_CONTACTS_3M',
 'IND_NBR_CONTACTS_6M',
 'IND_NBR_CONTACTS_9M',
 'IND_NBR_CONTACTS_12M',
 'IND_NBR_MOS_SINCE_LAST_MAIL',
 'HH_NBR_CONTACTS_3M',
 'HH_NBR_CONTACTS_6M',
 'HH_NBR_CONTACTS_9M',
 'HH_NBR_CONTACTS_12M',
 'HH_NBR_MOS_SINCE_LAST_MAIL'
         ]
len(rf_lst)

10

In [83]:
main = ls+rf_lst+prod+adlst
len(main)

344

In [84]:
main = list(set(main))
len(main)

318

In [85]:
main.append('RESPONSE_CNT')
len(main)

319

# shortlist the training data

In [86]:
df1 = df[main].copy()
df1.shape

(220658, 319)

# Branch 3 

# remove CHAZ vars and introduce hig score and removed age | birth vars

In [87]:
ls = [
'TU_CHAZ04',
    'TU_CHAZ09',
'AARP_AGE',
    'TU_INFERRED_AGE',
    'TU_BIRTH_DT_d',
    'DS_AU02D',
    'AARP_MEM_STATUS'

]
len(ls)

7

In [88]:
for i in ls:
    if i in df1.columns:
        print(i)
        df1.drop(columns=i,axis=1,inplace=True)
df1.shape

AARP_AGE
TU_INFERRED_AGE
TU_BIRTH_DT_d
DS_AU02D
AARP_MEM_STATUS


(220658, 314)

In [89]:
df1.shape

(220658, 314)

# Scalling the train data

In [90]:
# from sklearn.preprocessing import StandardScaler , MinMaxScaler , RobustScaler
# scale = StandardScaler()
# # scale = MinMaxScaler()
# # scale = RobustScaler()

# df1 = pd.DataFrame(scale.fit_transform(df1[df1.columns[~df1.columns.isin(['RESPONSE_CNT'])]].values),
#              columns=df1.columns[~df1.columns.isin(['RESPONSE_CNT'])])
# df1.shape
# df1 = pd.concat([df1,df[mls],df['RESPONSE_CNT'],df[adlst]],axis=1)
# df1.shape

In [91]:
df1 = pd.concat([df1,df[mls],df['SEQ_NO']], axis=1)
df1.shape

(220658, 318)

In [92]:
# drop the duplicate columns
df1 = df1.loc[:,~df1.columns.duplicated()].copy()
df1.shape

(220658, 318)

In [93]:
df1['SP_EMAILABLE_IND'] = df1['SP_EMAILABLE_IND'].fillna(0)
df1.shape

df1['AARP_SECONDARY_MEMBERSHIP_IND'] = df1['AARP_SECONDARY_MEMBERSHIP_IND'].fillna(0)
df1.shape

(220658, 318)

# shortlist the test data

In [94]:
dftest1 = dftest[main].copy()
dftest1.shape

(94567, 319)

# Branch 3 

# remove CHAZ vars and introduce hig score and removed age | birth vars

In [95]:
ls = [
'TU_CHAZ04',
    'TU_CHAZ09',
'AARP_AGE',
    'TU_INFERRED_AGE',
    'TU_BIRTH_DT_d',
    'DS_AU02D',
    'AARP_MEM_STATUS'

]
len(ls)

7

In [96]:
for i in ls:
    if i in dftest1.columns:
        print(i)
        dftest1.drop(columns=i,axis=1,inplace=True)
dftest1.shape

AARP_AGE
TU_INFERRED_AGE
TU_BIRTH_DT_d
DS_AU02D
AARP_MEM_STATUS


(94567, 314)

# Scalling the test data

In [9]:
# import pickle
# scalername = "standardscaler.pkl"  

# # load model from pickle file
# with open(scalername, 'rb') as file:  
#     scale = pickle.load(file)
# scale

# dftest1 = pd.DataFrame(scale.transform(dftest1[dftest1.columns[~dftest1.columns.isin(['RESPONSE_CNT'])]].values),
#              columns=dftest1.columns[~dftest1.columns.isin(['RESPONSE_CNT'])])
# dftest1.shape
# dftest1 = pd.concat([dftest1,dftest[mls] ,dftest['RESPONSE_CNT'],dftest[adlst]],axis=1)
# dftest1.shape

In [98]:
dftest1 = pd.concat([dftest1,dftest[mls],dftest['SEQ_NO']],axis=1)
dftest1.shape

(94567, 318)

In [99]:
dftest1 = dftest1.loc[:,~dftest1.columns.duplicated()].copy()
dftest1.shape

(94567, 318)

In [100]:
dftest1['SP_EMAILABLE_IND'] = dftest1['SP_EMAILABLE_IND'].fillna(0)
dftest1.shape

dftest1['AARP_SECONDARY_MEMBERSHIP_IND'] = dftest1['AARP_SECONDARY_MEMBERSHIP_IND'].fillna(0)
dftest1.shape

(94567, 318)

# Save the above data in train and test modelling

In [101]:
# df1.to_pickle('dataset/train_modelling_seqno.pkl')
# df1.shape

In [2]:
# dftest1.to_pickle('dataset/test_modelling_seqno.pkl')
# dftest1.shape

# Load the train and test modelling data

In [3]:
gm_append= pd.read_csv('/mnt/data/prod/hig/projects/models/model_build_2023/GM_model_all_vars_appended.csv')
gm_append.columns=map(str.upper, gm_append.columns)
gm_append.shape

(852236, 327)

In [4]:
df1 = pd.read_pickle('dataset/train_modelling_seqno.pkl')
df1.shape

(220658, 64)

In [5]:
dftest1 = pd.read_pickle('dataset/test_modelling_seqno.pkl')
dftest1.shape

(94567, 64)

In [12]:
df1 = pd.merge(df1,df[['SEQ_NO','state']],on='SEQ_NO',how='left')
df1.shape

(220658, 65)

In [13]:
dftest1 = pd.merge(dftest1,dftest[['SEQ_NO','state']],on='SEQ_NO',how='left')
dftest1.shape

(94567, 65)

In [6]:
df1 = pd.merge(df1,gm_append[['SEQ_NO','SCORE5']],on='SEQ_NO',how='left')
df1.shape

(220658, 65)

In [7]:
dftest1 = pd.merge(dftest1,gm_append[['SEQ_NO','SCORE5']],on='SEQ_NO',how='left')
dftest1.shape

(94567, 65)

In [9]:
df1 = df1[df1['SCORE5'].notna()]
df1.shape

(219780, 65)

In [10]:
dftest1 = dftest1[dftest1['SCORE5'].notna()]
dftest1.shape

(94207, 65)

In [16]:
# ls = ['TU_NUM_INS_INQ_0TO21M']
# for i in ls:
#     if i in df1.columns:
#         df1.drop(columns=i,axis=1,inplace=True)
#         dftest1.drop(columns=i,axis=1,inplace=True)
# print(df1.shape)
# print(dftest1.shape)

# Make them in train and test

In [11]:
# X_train = df1.drop('RESPONSE_CNT',axis=1)
X_train = df1.copy()
y_train = df1['RESPONSE_CNT']
X_train.shape , y_train.shape

((219780, 65), (219780,))

In [12]:
# X_test = dftest1.drop('RESPONSE_CNT',axis=1)
X_test = dftest1.copy()
y_test = dftest1['RESPONSE_CNT']
X_test.shape , y_test.shape

((94207, 65), (94207,))

In [13]:
X_train.shape , X_test.shape

((219780, 65), (94207, 65))

# column sequence

In [14]:
colseq = [
    'TU_MATCH_IND', #p1
 'SP_EMAILABLE_IND', #p2
 'AARP_SECONDARY_MEMBERSHIP_IND', #p1
 'TU_AUTO_XDT_d_st', #p2
 'TU_UNKNOWN_XDT_d_st', #p2
 'AARP_IBX_TOT_ONLINE_DOLLARS_st', #p1
 'IND_NBR_MOS_SINCE_LAST_MAIL',
 'IND_NBR_CONTACTS_3M',
 'HH_NBR_CONTACTS_3M',
 'IND_NBR_CONTACTS_12M',
 'IND_NBR_CONTACTS_9M', #p2
 'IND_NBR_CONTACTS_6M',
 'HH_NBR_CONTACTS_6M',
 'HH_NBR_MOS_SINCE_LAST_MAIL',
 'HH_NBR_CONTACTS_12M',
 'HH_NBR_CONTACTS_9M',
 'TU_UNKNOWN_XDT_d',
 'AARP_PRIMARY_BIRTH_DT_d',
 'TU_AUTO_XDT_d',
 'AARP_ADDR_MOVE_DT_d',
 'AARP_PAID_THROUGH_DT_d',
 'AARP_MEM_ORIGIN_DT_d',
 'TU_BCARD_ACCTS_OPEN_0TO24M', #p2
 'TU_BCARD_ACCTS_MOS_RCNT_OPEN',
 'TU_PUBREC_TRDLNE_DEROG_GT100DOL', #p2
 'AARP_MEM_TERM_MO_QTY', 
 'AARP_IBX_TOT_ONLINE_DOLLARS',
 'AARP_NUM_HISTPRT_OVERALL_AGG_HH',
 'AARP_IBX_NETWORTH_PR',
 'TU_ACCTS_MOS_OLD_OPEN',
 'DS_MD006',
 'DS_MD002',
 'AARP_NUM_CTCT_OVERALL_12M_AGG_HH',
 'AARP_NUM_HISTPRT_LFSTYL_AGG_HH', #p1
 'DS_MS530',
 'DS_XB831', #p2
 'DS_MS496',
 'AARP_LIFESTAGE_SEGMENT', #p2
 'TU_HIGSCORE_SHFD01', #removing HIG score
 'TU_PBCARD_ACCTS_AVG_BAL',
 'AARP_NMAS_44_CELL', #p2
 'AARP_IBX_TRND4TELCOM_OPTCALL_SVC',
 'TU_AVG_MOS_ACCTS_ON_FILE',
 'AARP_NUM_HISTPRT_FINAN_AGG_HH',
 'DS_MS545',
 'DS_MV028', #p2
 'AARP_IBX_HM_OWNER_RENTER_PR', #p1
 'AARP_IBX_PERSONIC_CLUSTER', 
 'DS_DF004', #p1
 'DS_MS544',
 'TU_NUM_INS_INQ_0TO24M',
 'DS_MS410',
 'DS_MS543',
 'DS_XB977',
#  'TU_NUM_INS_INQ_0TO21M', #p1
 'AARP_NUM_CURPRT_OVERALL_AGG_HH',
 'AARP_IBX_TOT_ONLINE_PURCHASES', #p2
 'AARP_PRIMARY_MEM_IND',
 'AARP_IBX_HM_ROOF_TYPE', #p2
#  'DS_XB875', # cbsa code 
 'RRI',
 'CRI',
    
    
    
]
len(colseq)

60

In [15]:
X_train[colseq].head()

TU_MATCH_IND  SP_EMAILABLE_IND  AARP_SECONDARY_MEMBERSHIP_IND  \
0             0               0.0                            0.0   
1             1               0.0                            0.0   
2             1               1.0                            1.0   
3             1               0.0                            0.0   
4             0               1.0                            0.0   

   TU_AUTO_XDT_d_st  TU_UNKNOWN_XDT_d_st  AARP_IBX_TOT_ONLINE_DOLLARS_st  \
0                 1                    1                               0   
1                 0                    0                               1   
2                 0                    0                               1   
3                 1                    1                               0   
4                 1                    1                               0   

   IND_NBR_MOS_SINCE_LAST_MAIL  IND_NBR_CONTACTS_3M  HH_NBR_CONTACTS_3M  \
0                          0.0                  4.0                 4.0   
1                          0.0                  1.0                 1.0   
2                          0.0                  5.0                 5.0   
3                          1.0                  3.0                 3.0   
4                          0.0                  5.0                 5.0   

   IND_NBR_CONTACTS_12M  ...  TU_NUM_INS_INQ_0TO24M  DS_MS410  DS_MS543  \
0                   7.0  ...                    0.0 -0.057457  255332.0   
1                   1.0  ...                    1.0  0.521739  280681.0   
2                  15.0  ...                    0.0 -0.057457  320138.0   
3                   3.0  ...                    0.0 -0.057457  197187.0   
4                   5.0  ...                    0.0 -0.057457  184166.0   

   DS_XB977  AARP_NUM_CURPRT_OVERALL_AGG_HH  AARP_IBX_TOT_ONLINE_PURCHASES  \
0 -0.080860                             1.0                            1.0   
1 -0.080860                             0.0                            1.0   
2 -0.080860                             0.0                            1.0   
3 -0.080860                             0.0                            1.0   
4  0.420497                             0.0                            1.0   

   AARP_PRIMARY_MEM_IND  AARP_IBX_HM_ROOF_TYPE       RRI       CRI  
0               0.10396              -0.064294  0.359366  1.205984  
1               0.10396              -0.064294  1.344722  0.839397  
2              -0.22805               0.030804  0.476290  1.349419  
3               0.10396              -0.064294  1.548966  0.689130  
4               0.10396               0.263138  1.537114  0.698633  

[5 rows x 60 columns]

In [16]:
X_test[colseq].head()

TU_MATCH_IND  SP_EMAILABLE_IND  AARP_SECONDARY_MEMBERSHIP_IND  \
0             0               0.0                            0.0   
1             1               1.0                            0.0   
2             1               0.0                            0.0   
3             1               0.0                            0.0   
4             1               0.0                            0.0   

   TU_AUTO_XDT_d_st  TU_UNKNOWN_XDT_d_st  AARP_IBX_TOT_ONLINE_DOLLARS_st  \
0                 1                    1                               0   
1                 0                    0                               0   
2                 0                    0                               0   
3                 0                    1                               0   
4                 0                    0                               0   

   IND_NBR_MOS_SINCE_LAST_MAIL  IND_NBR_CONTACTS_3M  HH_NBR_CONTACTS_3M  \
0                         24.0                  0.0                 0.0   
1                         24.0                  0.0                 0.0   
2                         10.0                  0.0                 0.0   
3                         24.0                  0.0                 0.0   
4                         24.0                  0.0                 0.0   

   IND_NBR_CONTACTS_12M  ...  TU_NUM_INS_INQ_0TO24M  DS_MS410  DS_MS543  \
0                   0.0  ...                    0.0 -0.861484  208242.0   
1                   0.0  ...                    2.0 -0.057457  583871.0   
2                   2.0  ...                    0.0 -0.057457  583116.0   
3                   0.0  ...                    1.0 -0.057457  287765.0   
4                   0.0  ...                    0.0 -0.057457  108983.0   

   DS_XB977  AARP_NUM_CURPRT_OVERALL_AGG_HH  AARP_IBX_TOT_ONLINE_PURCHASES  \
0 -0.080860                             0.0                            1.0   
1 -0.206808                             0.0                            3.0   
2 -0.080860                             0.0                            3.0   
3 -0.080860                             0.0                            4.0   
4 -0.080860                             0.0                            1.0   

   AARP_PRIMARY_MEM_IND  AARP_IBX_HM_ROOF_TYPE       RRI       CRI  
0               0.10396              -0.064294  1.291363  0.694481  
1               0.10396               0.063493  0.982598  0.987178  
2               0.10396              -0.064294  1.154619  0.571304  
3               0.10396               0.063493  1.318983  0.855122  
4               0.10396              -0.064294  0.932342  0.848599  

[5 rows x 60 columns]

# Automate the process

In [26]:
print('Please select the models from below list:\n')
print('''1. LogisticRegression
2. KNN 
3. RandomForest 
4. GradientBoosting 
5. XGBoost
6. Gaussian
7. DecisionTree
8. ExtraTree
9. Baggingclassifier
10. Adaboost
11. VotingClassifier
12. Neural Network MLP
13. LightGBM

''')


Please select the models from below list:

1. LogisticRegression
2. KNN 
3. RandomForest 
4. GradientBoosting 
5. XGBoost
6. Gaussian
7. DecisionTree
8. ExtraTree
9. Baggingclassifier
10. Adaboost
11. VotingClassifier
12. Neural Network MLP
13. LightGBM




In [116]:
X_train = X_train[colseq]
X_test = X_test[colseq]

In [121]:
X_train.shape , X_test.shape

((220658, 317), (94567, 317))

In [122]:
317/2

158.5

In [ ]:
global_count = 0
cols_removed_at_a_time = 2
drill_val = 159
var_list = []
auc_train_list = []
auc_test_list = []
precision = []
recall = []
shape_list = []
removed_var_list = []
train_df = pd.DataFrame()
test_df = pd.DataFrame()

accuracy_train_list = []
precision_train_list = []
recall_train_list = []
f1_score_train_list = []

accuracy_test_list = []
precision_test_list = []
recall_test_list = []
f1_score_test_list = []

# give the model number
ls = []
x = '13'
x = ls.append(x)
x = ls
x
##########################################################################################################
    # Modelling
for i in range(drill_val):
    print(' Process start for count -------------> ', i)
    model_list = []
    model_name = []
    model_score = []
    for i in x:
        if i =='1':
            model1 = LogisticRegression()
            print('Training start for ', str(model1)[:10])
            model1.fit(X_train , y_train)
            modell = model1.score(X_test , y_test)
            model_list.append(model1)
            model_name.append(str(model1)[:10])
            model_score.append(modell*100)
            print(str(model1)[:10],' model trained successfully.. with model score --> ',modell*100)
        elif i=='2':
            model2 = KNeighborsClassifier()
            print('Training start for ', model2)
            model2.fit(X_train , y_train)
            modelk = model2.score(X_test , y_test)
            model_list.append(model2)
            model_name.append(str(model2)[:10])
            model_score.append(modelk*100)
            print('KNN model trained successfully.. with model score --> ',modelk*100)
        elif i == '3':
            model3 = RandomForestClassifier()
            print('Training start for ', model3)
            model3.fit(X_train, y_train)
            modelr = model3.score(X_test , y_test)
            model_list.append(model3)
            model_name.append(str(model3)[:10])
            model_score.append(modelr*100)
            print('Random Forest model trained successfully.. with model score --> ',modelr*100)
        elif i == '4':
            model4 = GradientBoostingClassifier()
            print('Training start for ', model4)
            model4.fit(X_train, y_train)
            modelg = model4.score(X_test , y_test)
            model_list.append(model4)
            model_name.append(str(model4)[:10])
            model_score.append(modelg*100)
            print('Gradient boosting model trained successfully.. with model score --> ',modelg*100)
        elif i == '5':
            model5 = XGBClassifier()
            print('Training start for ', model5)
            model5.fit(X_train,y_train)
            modelx = model5.score(X_test , y_test)
            model_list.append(model5)
            model_name.append(str(model5)[:10])
            model_score.append(modelx*100)
            print('XGBoost model trained successfully.. with model score --> ' ,modelx*100)
        elif i=='6':
            model6 = GaussianNB()
            print('Training start for ', model6)
            model6.fit(X_train , y_train)
            modelgnb = model6.score(X_test , y_test)
            model_list.append(model6)
            model_name.append(str(model6)[:10])
            model_score.append(modelgnb*100)
            print('Gaussian Naive Byers model trained successfully.. with model score --> ',modelgnb*100)
        elif i == '7':
            model7 = DecisionTreeClassifier()
            print('Training start for ', model7)
            model7.fit(X_train, y_train)
            modeldt = model7.score(X_test , y_test)
            model_list.append(model7)
            model_name.append(str(model7)[:10])
            model_score.append(modeldt*100)
            print('DecisionTree model trained successfully.. with model score --> ',modeldt*100)
        elif i == '8':
            model8 = ExtraTreesClassifier()
            print('Training start for ', model8)
            model8.fit(X_train, y_train)
            modelet = model8.score(X_test , y_test)
            model_list.append(model8)
            model_name.append(str(model8)[:10])
            model_score.append(modelet*100)
            print('ExtraTree Classifier model trained successfully.. with model score --> ',modelet*100)
        elif i == '9':
            model9=BaggingClassifier(DecisionTreeClassifier(),max_samples=0.5,max_features=1.0,n_estimators=10)
            print('Training start for ', model9)
            model9.fit(X_train,y_train)
            modelbc = model9.score(X_test , y_test)
            model_list.append(model9)
            model_name.append(str(model9)[:10])
            model_score.append(modelbc*100)
            print('BaggingClassifier model trained successfully.. with model score --> ' ,modelbc*100)
        elif i == '10':
    #         model10 = AdaBoostClassifier(DecisionTreeClassifier(min_samples_split=10,max_depth=4),
    #                                      n_estimators=60,learning_rate=0.3)
            model10 = AdaBoostClassifier(RandomForestClassifier(min_samples_split=10,max_depth=4),
                                         n_estimators=60,learning_rate=0.3)
            print('Training start for ', model10)
            model10.fit(X_train,y_train)
            modelab = model10.score(X_test , y_test)
            model_list.append(model10)
            model_name.append(str(model10)[:10])
            model_score.append(modelab*100)
            print('AdaBoost model trained successfully.. with model score --> ' ,modelab*100)
        elif i == '11':
            model11=VotingClassifier(estimators=[
                                            ('lgbm',LGBMClassifier()),
                                             ('rf',RandomForestClassifier()),
                                             ('gbm',GradientBoostingClassifier()),
                                             ('xgb',XGBClassifier())],voting='soft')
            print('Training start for ', model11)
            model11.fit(X_train,y_train)
            modelvc = model11.score(X_test , y_test)
            model_list.append(model11)
            model_name.append(str(model11)[:10])
            model_score.append(modelvc*100)
            print('Voting trained successfully.. with model score --> ' ,modelvc*100)
        elif i == '12':
            model12 = MLPClassifier(hidden_layer_sizes=(8,8,8), activation='relu', solver='adam',
                                    max_iter=200,verbose=3,
                                   early_stopping=True)
            print('Training start for ', model12)
            model12.fit(X_train,y_train)
            modelnn = model12.score(X_test , y_test)
            model_list.append(model12)
            model_name.append(str(model12)[:10])
            model_score.append(modelnn*100)
            print('Multi Layered Perceptron model trained successfully.. with model score --> ' ,modelnn*100)
        elif i == '13':
            model13=LGBMClassifier()
            print('Training start for ', model13)
            model13.fit(X_train,y_train)
            modellgbm = model13.score(X_test , y_test)
            model_list.append(model13)
            model_name.append(str(model13)[:10])
            model_score.append(modellgbm*100)
            print('LightGBM trained successfully.. with model score --> ' ,modellgbm*100)

##########################################################################################################

# #define metrics
# plt.figure(figsize=(12,8))
    for i in model_list:
        y_pred_proba = i.predict_proba(X_train)[::,1]
        fpr, tpr, _ = metrics.roc_curve(y_train,  y_pred_proba)
        auc = metrics.roc_auc_score(y_train, y_pred_proba)
    #create ROC curve
#     plt.plot(fpr,tpr,label=str(i)[:5]+" AUC="+str(round(auc,2)))
#     plt.ylabel('True Positive Rate')
#     plt.xlabel('False Positive Rate')
#     plt.legend(loc=4)
    global_auc_train =round(auc,6)
#     plt.show()


# #define metrics
# plt.figure(figsize=(12,8))
    for i in model_list:
        y_pred_proba = i.predict_proba(X_test)[::,1]
        fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
        auc = metrics.roc_auc_score(y_test, y_pred_proba)
    #create ROC curve
#     plt.plot(fpr,tpr,label=str(i)[:5]+" AUC="+str(round(auc,2)))
#     plt.ylabel('True Positive Rate')
#     plt.xlabel('False Positive Rate')
#     plt.legend(loc=4)
    global_auc_test =round(auc,6)
#     plt.show()




##########################################################################################################

    # Varibale importance for the trained models

    var_imp = pd.DataFrame()
    ls = []
    for i in model_list:
        t = pd.DataFrame.from_dict({'VAR':X_train.columns,
                                          'IMP':i.feature_importances_}).sort_values('IMP',ascending=False)
        t = t.reset_index()
        t.drop('index',axis=1,inplace=True)
        l = list(t.iloc[:10]['VAR'])
        ls = ls + l
        t['model_name'] = str(i)[:10]
        var_imp = pd.concat([var_imp,t])

    removed_var_list = list(var_imp.iloc[range(len(var_imp)-cols_removed_at_a_time,len(var_imp))]['VAR'])
    removed_var = ''
    for i in removed_var_list[::-1]:
        removed_var = removed_var+i+', '

##########################################################################################################

    # train deciles

    temp = pd.DataFrame()
    count = 0
    for i in model_list:
        # Initialize the model here
        model = i
        type(model)
        
        predictors=X_train
        test_predictors=X_train ######################

        predictors.shape

        eval_set=[(predictors,y_train)]

        #Predicting train for testing
        predict_train = model.predict(predictors)
#         print('\nTarget on train data',predict_train)
        accuracy_train = accuracy_score(y_train,predict_train)
#         print('\naccuracy_score on train dataset : ', accuracy_train)

        cmxg = metrics.confusion_matrix(y_train,predict_train)
#         print(cmxg)

        #Probability score between 1 & 0
        proba=model.predict_proba(predictors)
#         print(proba)

        #classification_report
#         print(classification_report(y_train, predict_train))
        #mean_squared_error
#         print(np.sqrt(metrics.mean_squared_error(y_train,predict_train)))
        auc = roc_auc_score(y_train, predict_train)
#         print('\nroc_auc_score on train dataset : ', auc)

        predict_test = model.predict(test_predictors)
#         print('\nTarget on test data',predict_test)
        accuracy_test = accuracy_score(y_train,predict_test)####################
#         print('\naccuracy_score on test dataset : ', accuracy_test)

        cmxgt = metrics.confusion_matrix(y_train,predict_test)####################
#         print(cmxgt)
        auc_test = roc_auc_score(y_train,predict_test)####################
#         print('\nroc_auc_score on test dataset : ', auc_test)
        #classification_report
#         print(classification_report(y_train, predict_test))####################
        #mean_squared_error
#         print(np.sqrt(metrics.mean_squared_error(y_train,predict_test)))#####################

        test_predictors["Pred_prob"] = model.predict_proba(test_predictors)[:,1]

        test_predictors['decile'] = pd.qcut(test_predictors['Pred_prob'].rank(method='first').values, 10, duplicates='drop').codes + 1

        new_actual = 'Actual'+'('+str(X_train.shape[1]-2)+')'
        test_predictors[new_actual]  = y_train #######################

        DF0 = test_predictors.groupby(['decile'], as_index=False).agg({'Pred_prob': 'count',new_actual: 'sum'})
        DF0


        DF0[new_actual] =  list(DF0[new_actual][::-1])
    
        DF0['% mail'] = DF0['Pred_prob'].cumsum()/DF0['Pred_prob'].sum()
        DF0['% mail'] = round(DF0['% mail'],4)*100

        new_resp = 'cumm resp %'+'('+str(X_train.shape[1]-3)+')'
        DF0[new_resp] = DF0[new_actual].cumsum()/DF0[new_actual].sum()
        DF0[new_resp] = round(DF0[new_resp],4)*100
        
        new_cum = 'cumm RR'+'('+str(X_train.shape[1]-3)+')'
        DF0[new_cum]  = DF0[new_actual].cumsum()/DF0['Pred_prob'].cumsum()
        DF0[new_cum] = round(DF0[new_cum],4)*100
        train_df = pd.concat([train_df,DF0],axis=1)
        
        
        X_train.drop(columns=['Pred_prob','decile',new_actual],inplace=True)###############################
#     print(temp.shape)

##########################################################################################################


    temp = pd.DataFrame()
    count = 0
    for i in model_list:

        # Initialize the model here
        model = i
        type(model)

        predictors=X_train
        test_predictors=X_test ######################

        predictors.shape

        eval_set=[(predictors,y_train)]

        #Predicting train for testing
        predict_train = model.predict(predictors)
#         print('\nTarget on train data',predict_train)
        accuracy_train = accuracy_score(y_train,predict_train)
#         print('\naccuracy_score on train dataset : ', accuracy_train)

        cmxg = metrics.confusion_matrix(y_train,predict_train)
#         print(cmxg)

        #Probability score between 1 & 0
        proba=model.predict_proba(predictors)
#         print(proba)

        #classification_report
#         print(classification_report(y_train, predict_train))
        #mean_squared_error
#         print(np.sqrt(metrics.mean_squared_error(y_train,predict_train)))
        auc = roc_auc_score(y_train, predict_train)
#         print('\nroc_auc_score on train dataset : ', auc)

        predict_test = model.predict(test_predictors)
#         print('\nTarget on test data',predict_test)
        accuracy_test = accuracy_score(y_test,predict_test)####################
#         print('\naccuracy_score on test dataset : ', accuracy_test)

        cmxgt = metrics.confusion_matrix(y_test,predict_test)####################
#         print(cmxgt)
        auc_test = roc_auc_score(y_test,predict_test)####################
#         print('\nroc_auc_score on test dataset : ', auc_test)
        #classification_report
#         print(classification_report(y_test, predict_test))####################
        #mean_squared_error
#         print(np.sqrt(metrics.mean_squared_error(y_test,predict_test)))#####################

        test_predictors["Pred_prob"] = model.predict_proba(test_predictors)[:,1]

        test_predictors['decile'] = pd.qcut(test_predictors['Pred_prob'].rank(method='first').values, 10, duplicates='drop').codes + 1

        new_actual = 'Actual'+'('+str(X_test.shape[1]-2)+')'
        test_predictors[new_actual]  = y_test #######################

        DF0 = test_predictors.groupby(['decile'], as_index=False).agg({'Pred_prob': 'count',new_actual: 'sum'})
        DF0


        DF0[new_actual] =  list(DF0[new_actual][::-1])
    
        DF0['% mail'] = DF0['Pred_prob'].cumsum()/DF0['Pred_prob'].sum()
        DF0['% mail'] = round(DF0['% mail'],4)*100

        new_resp = 'cumm resp %'+'('+str(X_test.shape[1]-3)+')'
        DF0[new_resp] = DF0[new_actual].cumsum()/DF0[new_actual].sum()
        DF0[new_resp] = round(DF0[new_resp],4)*100
        
        new_cum = 'cumm RR'+'('+str(X_test.shape[1]-3)+')'
        DF0[new_cum]  = DF0[new_actual].cumsum()/DF0['Pred_prob'].cumsum()
        DF0[new_cum] = round(DF0[new_cum],4)*100

        test_df = pd.concat([test_df,DF0],axis=1)
        
        X_test.drop(columns=['Pred_prob','decile',new_actual],inplace=True)###############################
#     print(temp.shape)
    
##########################################################################################################

    y_preds_train = model.predict(X_train)
    accuracy_train = accuracy_score(y_train,y_preds_train)
    precision_train = precision_score(y_train,y_preds_train)
    recall_train = recall_score(y_train,y_preds_train)
    f1_score_train = f1_score(y_train,y_preds_train)
    
    
    y_preds_test = model.predict(X_test)
    accuracy_test = accuracy_score(y_test,y_preds_test)
    precision_test = precision_score(y_test,y_preds_test)
    recall_test = recall_score(y_test,y_preds_test)
    f1_score_test = f1_score(y_test,y_preds_test)
    
    
    shape_list.append(X_train.shape[1])
    auc_train_list.append(global_auc_train)
    auc_test_list.append(global_auc_test)
    var_list.append(removed_var) #kunal
    
    accuracy_train_list.append(accuracy_train)
    precision_train_list.append(precision_train)
    recall_train_list.append(recall_train)
    f1_score_train_list.append(f1_score_train)

    
    accuracy_test_list.append(accuracy_test)
    precision_test_list.append(precision_test)
    recall_test_list.append(recall_test)
    f1_score_test_list.append(f1_score_test)

##########################################################################################################

    for i in removed_var_list:
        if i in X_train.columns:
            if X_train.shape[1]>2:
                X_train.drop(i,axis=1,inplace=True)
                X_test.drop(i,axis=1,inplace=True)

            
##########################################################################################################

dfnew = pd.DataFrame()
dfnew['no of cols'] = shape_list
dfnew['removed_vars'] = var_list #kunal
dfnew['precision_train'] = precision_train_list
dfnew['precision_test'] = precision_test_list
dfnew['recall_train'] = recall_train_list
dfnew['recall_test'] = recall_test_list
dfnew['f1_score_train'] = f1_score_train_list
dfnew['f1_score_test'] = f1_score_test_list
dfnew['auc_train'] = auc_train_list
dfnew['auc_test'] = auc_test_list
dfnew['train_acc_score'] = accuracy_train_list
dfnew['test_acc_score']  = accuracy_test_list
print(dfnew.shape)



 Process start for count ------------->  0
Training start for  LGBMClassifier()
LightGBM trained successfully.. with model score -->  100.0
 Process start for count ------------->  1
Training start for  LGBMClassifier()
LightGBM trained successfully.. with model score -->  100.0
 Process start for count ------------->  2
Training start for  LGBMClassifier()
LightGBM trained successfully.. with model score -->  100.0
 Process start for count ------------->  3
Training start for  LGBMClassifier()
LightGBM trained successfully.. with model score -->  100.0
 Process start for count ------------->  4
Training start for  LGBMClassifier()


In [ ]:
train_df.shape , test_df.shape

In [ ]:
train_df = train_df.loc[:,~train_df.columns.duplicated()].copy()

test_df = test_df.loc[:,~test_df.columns.duplicated()].copy()

In [ ]:
train_df.shape , test_df.shape

In [123]:
dfnew.to_csv('drilled_results.csv',index=False)
dfnew.shape

(39, 12)

In [124]:
train_df.to_csv('train_df.csv',index=False)
train_df.shape

(10, 120)

In [125]:
test_df.to_csv('test_df.csv',index=False)
test_df.shape

(10, 120)

# Modelling

In [34]:
print('Please select the models from below list:\n')
print('''1. LogisticRegression
2. KNN 
3. RandomForest 
4. GradientBoosting 
5. XGBoost
6. Gaussian
7. DecisionTree
8. ExtraTree
9. Baggingclassifier
10. Adaboost
11. VotingClassifier
12. Neural Network MLP
13. LightGBM

''')
x = str(input('\nenter the model numbers with space..'))

Please select the models from below list:

1. LogisticRegression
2. KNN 
3. RandomForest 
4. GradientBoosting 
5. XGBoost
6. Gaussian
7. DecisionTree
8. ExtraTree
9. Baggingclassifier
10. Adaboost
11. VotingClassifier
12. Neural Network MLP
13. LightGBM





enter the model numbers with space.. 3 4 5 7 8 9 10 11 12 13


In [35]:
x = x.split(' ')
type(x)

x

['3', '4', '5', '7', '8', '9', '10', '11', '12', '13']

In [36]:
X_train.shape , y_train.shape

((220658, 64), (220658,))

In [37]:
X_train = X_train[colseq].copy()
X_test = X_test[colseq].copy()

In [38]:
X_train.shape , X_test.shape

((220658, 59), (94567, 59))

In [39]:
X_train.head()

TU_MATCH_IND  SP_EMAILABLE_IND  AARP_SECONDARY_MEMBERSHIP_IND  \
0             0               0.0                            0.0   
1             1               0.0                            0.0   
2             1               1.0                            1.0   
3             1               0.0                            0.0   
4             0               1.0                            0.0   

   TU_AUTO_XDT_d_st  TU_UNKNOWN_XDT_d_st  AARP_IBX_TOT_ONLINE_DOLLARS_st  \
0                 1                    1                               0   
1                 0                    0                               1   
2                 0                    0                               1   
3                 1                    1                               0   
4                 1                    1                               0   

   IND_NBR_MOS_SINCE_LAST_MAIL  IND_NBR_CONTACTS_3M  HH_NBR_CONTACTS_3M  \
0                          0.0                  4.0                 4.0   
1                          0.0                  1.0                 1.0   
2                          0.0                  5.0                 5.0   
3                          1.0                  3.0                 3.0   
4                          0.0                  5.0                 5.0   

   IND_NBR_CONTACTS_12M  ...  TU_NUM_INS_INQ_0TO24M  DS_MS410  DS_MS543  \
0                   7.0  ...                    0.0 -0.057457  255332.0   
1                   1.0  ...                    1.0  0.521739  280681.0   
2                  15.0  ...                    0.0 -0.057457  320138.0   
3                   3.0  ...                    0.0 -0.057457  197187.0   
4                   5.0  ...                    0.0 -0.057457  184166.0   

   DS_XB977  AARP_NUM_CURPRT_OVERALL_AGG_HH  AARP_IBX_TOT_ONLINE_PURCHASES  \
0 -0.080860                             1.0                            1.0   
1 -0.080860                             0.0                            1.0   
2 -0.080860                             0.0                            1.0   
3 -0.080860                             0.0                            1.0   
4  0.420497                             0.0                            1.0   

   AARP_PRIMARY_MEM_IND  AARP_IBX_HM_ROOF_TYPE       RRI       CRI  
0               0.10396              -0.064294  0.359366  1.205984  
1               0.10396              -0.064294  1.344722  0.839397  
2              -0.22805               0.030804  0.476290  1.349419  
3               0.10396              -0.064294  1.548966  0.689130  
4               0.10396               0.263138  1.537114  0.698633  

[5 rows x 59 columns]

In [40]:
X_test.head()

TU_MATCH_IND  SP_EMAILABLE_IND  AARP_SECONDARY_MEMBERSHIP_IND  \
0             0               0.0                            0.0   
1             1               1.0                            0.0   
2             1               0.0                            0.0   
3             1               0.0                            0.0   
4             1               0.0                            0.0   

   TU_AUTO_XDT_d_st  TU_UNKNOWN_XDT_d_st  AARP_IBX_TOT_ONLINE_DOLLARS_st  \
0                 1                    1                               0   
1                 0                    0                               0   
2                 0                    0                               0   
3                 0                    1                               0   
4                 0                    0                               0   

   IND_NBR_MOS_SINCE_LAST_MAIL  IND_NBR_CONTACTS_3M  HH_NBR_CONTACTS_3M  \
0                         24.0                  0.0                 0.0   
1                         24.0                  0.0                 0.0   
2                         10.0                  0.0                 0.0   
3                         24.0                  0.0                 0.0   
4                         24.0                  0.0                 0.0   

   IND_NBR_CONTACTS_12M  ...  TU_NUM_INS_INQ_0TO24M  DS_MS410  DS_MS543  \
0                   0.0  ...                    0.0 -0.861484  208242.0   
1                   0.0  ...                    2.0 -0.057457  583871.0   
2                   2.0  ...                    0.0 -0.057457  583116.0   
3                   0.0  ...                    1.0 -0.057457  287765.0   
4                   0.0  ...                    0.0 -0.057457  108983.0   

   DS_XB977  AARP_NUM_CURPRT_OVERALL_AGG_HH  AARP_IBX_TOT_ONLINE_PURCHASES  \
0 -0.080860                             0.0                            1.0   
1 -0.206808                             0.0                            3.0   
2 -0.080860                             0.0                            3.0   
3 -0.080860                             0.0                            4.0   
4 -0.080860                             0.0                            1.0   

   AARP_PRIMARY_MEM_IND  AARP_IBX_HM_ROOF_TYPE       RRI       CRI  
0               0.10396              -0.064294  1.291363  0.694481  
1               0.10396               0.063493  0.982598  0.987178  
2               0.10396              -0.064294  1.154619  0.571304  
3               0.10396               0.063493  1.318983  0.855122  
4               0.10396              -0.064294  0.932342  0.848599  

[5 rows x 59 columns]

In [41]:
# np.random.seed(42)
model_list = []
model_name = []
model_score = []
for i in x:
    if i =='1':
        model1 = LogisticRegression()
        print('Training start for ', str(model1)[:10])
        model1.fit(X_train , y_train)
        modell = model1.score(X_test , y_test)
        model_list.append(model1)
        model_name.append(str(model1)[:10])
        model_score.append(modell*100)
        print(str(model1)[:10],' model trained successfully.. with model score --> ',modell*100)
    elif i=='2':
        model2 = KNeighborsClassifier()
        print('Training start for ', model2)
        model2.fit(X_train , y_train)
        modelk = model2.score(X_test , y_test)
        model_list.append(model2)
        model_name.append(str(model2)[:10])
        model_score.append(modelk*100)
        print('KNN model trained successfully.. with model score --> ',modelk*100)
    elif i == '3':
        model3 = RandomForestClassifier()
        print('Training start for ', model3)
        model3.fit(X_train, y_train)
        modelr = model3.score(X_test , y_test)
        model_list.append(model3)
        model_name.append(str(model3)[:10])
        model_score.append(modelr*100)
        print('Random Forest model trained successfully.. with model score --> ',modelr*100)
    elif i == '4':
        model4 = GradientBoostingClassifier()
        print('Training start for ', model4)
        model4.fit(X_train, y_train)
        modelg = model4.score(X_test , y_test)
        model_list.append(model4)
        model_name.append(str(model4)[:10])
        model_score.append(modelg*100)
        print('Gradient boosting model trained successfully.. with model score --> ',modelg*100)
    elif i == '5':
        model5 = XGBClassifier()
        print('Training start for ', model5)
        model5.fit(X_train,y_train)
        modelx = model5.score(X_test , y_test)
        model_list.append(model5)
        model_name.append(str(model5)[:10])
        model_score.append(modelx*100)
        print('XGBoost model trained successfully.. with model score --> ' ,modelx*100)
    elif i=='6':
        model6 = GaussianNB()
        print('Training start for ', model6)
        model6.fit(X_train , y_train)
        modelgnb = model6.score(X_test , y_test)
        model_list.append(model6)
        model_name.append(str(model6)[:10])
        model_score.append(modelgnb*100)
        print('Gaussian Naive Byers model trained successfully.. with model score --> ',modelgnb*100)
    elif i == '7':
        model7 = DecisionTreeClassifier()
        print('Training start for ', model7)
        model7.fit(X_train, y_train)
        modeldt = model7.score(X_test , y_test)
        model_list.append(model7)
        model_name.append(str(model7)[:10])
        model_score.append(modeldt*100)
        print('DecisionTree model trained successfully.. with model score --> ',modeldt*100)
    elif i == '8':
        model8 = ExtraTreesClassifier()
        print('Training start for ', model8)
        model8.fit(X_train, y_train)
        modelet = model8.score(X_test , y_test)
        model_list.append(model8)
        model_name.append(str(model8)[:10])
        model_score.append(modelet*100)
        print('ExtraTree Classifier model trained successfully.. with model score --> ',modelet*100)
    elif i == '9':
        model9=BaggingClassifier(DecisionTreeClassifier(),max_samples=0.5,max_features=1.0,n_estimators=10)
        print('Training start for ', model9)
        model9.fit(X_train,y_train)
        modelbc = model9.score(X_test , y_test)
        model_list.append(model9)
        model_name.append(str(model9)[:10])
        model_score.append(modelbc*100)
        print('BaggingClassifier model trained successfully.. with model score --> ' ,modelbc*100)
    elif i == '10':
#         model10 = AdaBoostClassifier(DecisionTreeClassifier(min_samples_split=10,max_depth=4),
#                                      n_estimators=60,learning_rate=0.3)
        model10 = AdaBoostClassifier(RandomForestClassifier(min_samples_split=10,max_depth=4),
                                     n_estimators=60,learning_rate=0.3)
        print('Training start for ', model10)
        model10.fit(X_train,y_train)
        modelab = model10.score(X_test , y_test)
        model_list.append(model10)
        model_name.append(str(model10)[:10])
        model_score.append(modelab*100)
        print('AdaBoost model trained successfully.. with model score --> ' ,modelab*100)
    elif i == '11':
        model11=VotingClassifier(estimators=[
#                                             ('etc',ExtraTreesClassifier()),
#                                          ('dtc',DecisionTreeClassifier()),
#                                          ('lr',LogisticRegression()),
                                        ('lgbm',LGBMClassifier()),
                                         ('rf',RandomForestClassifier()),
                                         ('gbm',GradientBoostingClassifier()),
                                         ('xgb',XGBClassifier())],voting='soft')
        print('Training start for ', model11)
        model11.fit(X_train,y_train)
        modelvc = model11.score(X_test , y_test)
        model_list.append(model11)
        model_name.append(str(model11)[:10])
        model_score.append(modelvc*100)
        print('Voting trained successfully.. with model score --> ' ,modelvc*100)
    elif i == '12':
        model12 = MLPClassifier(hidden_layer_sizes=(64,64,64), activation='relu', solver='adam',
                                max_iter=200,verbose=3,
                               early_stopping=True)
        print('Training start for ', model12)
        model12.fit(X_train,y_train)
        modelnn = model12.score(X_test , y_test)
        model_list.append(model12)
        model_name.append(str(model12)[:10])
        model_score.append(modelnn*100)
        print('Multi Layered Perceptron model trained successfully.. with model score --> ' ,modelnn*100)
    elif i == '13':
        model13=LGBMClassifier()
        print('Training start for ', model13)
        model13.fit(X_train,y_train)
        modellgbm = model13.score(X_test , y_test)
        model_list.append(model13)
        model_name.append(str(model13)[:10])
        model_score.append(modellgbm*100)
        print('LightGBM trained successfully.. with model score --> ' ,modellgbm*100)




Training start for  RandomForestClassifier()
Random Forest model trained successfully.. with model score -->  82.17771527065467
Training start for  GradientBoostingClassifier()
Gradient boosting model trained successfully.. with model score -->  82.05187856228918
Training start for  XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=None, tree_method=None,
              validate_parameters=None, verbosity=None)
[06:55:32] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default e

In [42]:
X_train.shape , X_test.shape

((220658, 59), (94567, 59))

# LGBMclassifier

In [229]:
X_train[colseq].head()

HH_NBR_CONTACTS_3M  TU_AVG_MOS_ACCTS_ON_FILE  TU_CHAZ11  AARP_MEM_X_RENEW  \
0                 4.0                     127.0      830.0               5.0   
1                 1.0                     113.0      874.0               1.0   
2                 5.0                     118.0      745.0               4.0   
3                 3.0                     148.0      683.0               5.0   
4                 5.0                     127.0      830.0               0.0   

   IND_NBR_CONTACTS_12M  IND_NBR_MOS_SINCE_LAST_MAIL  \
0                   7.0                          0.0   
1                   1.0                          0.0   
2                  15.0                          0.0   
3                   3.0                          1.0   
4                   5.0                          0.0   

   AARP_NUM_HISTPRT_FINAN_AGG_HH  IND_NBR_CONTACTS_3M  IND_NBR_CONTACTS_6M  \
0                            1.0                  4.0                  7.0   
1                            0.0                  1.0                  1.0   
2                            0.0                  5.0                  8.0   
3                            0.0                  3.0                  3.0   
4                            1.0                  5.0                  5.0   

   DS_MD014  ...  DS_MS496  HH_NBR_CONTACTS_6M  HH_NBR_CONTACTS_9M  \
0      64.0  ...  0.047712                 7.0                 7.0   
1      55.0  ...  0.047712                 1.0                 1.0   
2      74.0  ...  0.047712                 8.0                11.0   
3      70.0  ... -0.295918                 3.0                 3.0   
4      68.0  ... -0.295918                 5.0                 5.0   

   AARP_NMAS_37_REN_OT  DS_MS530  TU_BCARD_ACCTS  \
0                 77.0      20.5             5.0   
1                 69.0      18.7             2.0   
2                 57.0      21.3            14.0   
3                 44.0      26.2             5.0   
4                 33.0      30.0             5.0   

   AARP_NUM_CURPRT_OVERALL_AGG_HH  DS_MS536  AARP_ADDR_MOVE_DT_d  DS_MD002  
0                             1.0      2.61                 66.0      79.0  
1                             0.0      1.81                 53.0     101.0  
2                             0.0      2.25                 87.0     239.0  
3                             0.0      2.39                188.0      89.0  
4                             0.0      1.78                 87.0      65.0  

[5 rows x 57 columns]

In [230]:
X_test[colseq].head()

HH_NBR_CONTACTS_3M  TU_AVG_MOS_ACCTS_ON_FILE  TU_CHAZ11  AARP_MEM_X_RENEW  \
0                 0.0                     127.0      830.0               1.0   
1                 0.0                     214.0      908.0               9.0   
2                 0.0                     114.0      772.0               9.0   
3                 0.0                     118.0      791.0               3.0   
4                 0.0                      94.0      799.0               4.0   

   IND_NBR_CONTACTS_12M  IND_NBR_MOS_SINCE_LAST_MAIL  \
0                   0.0                         24.0   
1                   0.0                         24.0   
2                   2.0                         10.0   
3                   0.0                         24.0   
4                   0.0                         24.0   

   AARP_NUM_HISTPRT_FINAN_AGG_HH  IND_NBR_CONTACTS_3M  IND_NBR_CONTACTS_6M  \
0                            0.0                  0.0                  0.0   
1                            0.0                  0.0                  0.0   
2                            0.0                  0.0                  0.0   
3                            0.0                  0.0                  0.0   
4                            0.0                  0.0                  0.0   

   DS_MD014  ...  DS_MS496  HH_NBR_CONTACTS_6M  HH_NBR_CONTACTS_9M  \
0      60.0  ...  0.047712                 0.0                 2.0   
1      51.0  ...  0.047712                 0.0                 0.0   
2      79.0  ...  0.047712                 0.0                 0.0   
3      39.0  ...  0.047712                 0.0                 0.0   
4      62.0  ... -0.331806                 0.0                 0.0   

   AARP_NMAS_37_REN_OT  DS_MS530  TU_BCARD_ACCTS  \
0                 45.0      13.7             5.0   
1                 12.0      14.6             5.0   
2                 78.0      19.9             7.0   
3                 13.0       4.2             2.0   
4                 41.0      15.3             7.0   

   AARP_NUM_CURPRT_OVERALL_AGG_HH  DS_MS536  AARP_ADDR_MOVE_DT_d  DS_MD002  
0                             0.0      3.58                 87.0      63.0  
1                             0.0      2.91                 87.0     142.0  
2                             0.0      3.23                 34.0     221.0  
3                             0.0      3.23                 48.0     198.0  
4                             0.0      2.96                 88.0     139.0  

[5 rows x 57 columns]

In [231]:
model13=LGBMClassifier(learning_rate =0.046,n_estimators= 430)
print('Training start for ', model13)
model13.fit(X_train[colseq],y_train)
modellgbm = model13.score(X_test[colseq] , y_test)
# model_list.append(model13)
# model_name.append(str(model13)[:10])
# model_score.append(modellgbm*100)
y_preds = model13.predict(X_test[colseq])
f1 = f1_score(y_test,y_preds)
print('LightGBM trained successfully.. with model score --> ' ,modellgbm*100)
print('LightGBM trained successfully.. with model f1_score --> ' ,f1*100,' vars-',len(colseq))




Training start for  LGBMClassifier(learning_rate=0.046, n_estimators=430)
LightGBM trained successfully.. with model score -->  82.8692884409995
LightGBM trained successfully.. with model f1_score -->  43.43970393128971  vars- 57


In [ ]:
# base model without HIG score
Training start for  LGBMClassifier()
LightGBM trained successfully.. with model score -->  82.67577484746265
LightGBM trained successfully.. with model f1_score -->  41.49137530802471  vars- 69

In [ ]:
# all 62 vars
Training start for  LGBMClassifier(learning_rate=0.046, n_estimators=370)
LightGBM trained successfully.. with model score -->  82.93167806951685
LightGBM trained successfully.. with model f1_score -->  43.462117762443526

Training start for  LGBMClassifier(learning_rate=0.046, n_estimators=370)
LightGBM trained successfully.. with model score -->  82.8682309896687
LightGBM trained successfully.. with model f1_score -->  43.27182324311076  vars- 61

Training start for  LGBMClassifier(learning_rate=0.046, n_estimators=370)
LightGBM trained successfully.. with model score -->  82.80478390982054
LightGBM trained successfully.. with model f1_score -->  42.94185760903891  vars- 60

Training start for  LGBMClassifier(learning_rate=0.046, n_estimators=430)
LightGBM trained successfully.. with model score -->  82.87457569765351
LightGBM trained successfully.. with model f1_score -->  43.44728847295457  vars- 60

In [ ]:
# adding 4 vars from all 1000+ vars list + tu_ims 0to21
Training start for  LGBMClassifier(learning_rate=0.046, n_estimators=370)
LightGBM trained successfully.. with model score -->  83.12413421172292
LightGBM trained successfully.. with model f1_score -->  44.4575923154561


# same but we do z score normalization
Training start for  LGBMClassifier(learning_rate=0.046, n_estimators=370)
LightGBM trained successfully.. with model score -->  82.9982975033574
LightGBM trained successfully.. with model f1_score -->  42.45114181401675

In [ ]:
# adding 4 vars from all 1000+ vars list
Training start for  LGBMClassifier(learning_rate=0.046, n_estimators=370)
LightGBM trained successfully.. with model score -->  83.06491693719796
LightGBM trained successfully.. with model f1_score -->  44.27433104840113

In [ ]:
# all 62 vars
Training start for  LGBMClassifier(learning_rate=0.046, n_estimators=370)
LightGBM trained successfully.. with model score -->  82.93167806951685
LightGBM trained successfully.. with model f1_score -->  43.462117762443526

In [ ]:
# lgbm less p2
Training start for  LGBMClassifier(learning_rate=0.046, n_estimators=520)
LightGBM trained successfully.. with model score -->  82.88409275963073
LightGBM trained successfully.. with model f1_score -->  43.25480297293507

In [ ]:
# lgbm less p1
Training start for  LGBMClassifier(learning_rate=0.046, n_estimators=520)
LightGBM trained successfully.. with model score -->  82.98878044138019
LightGBM trained successfully.. with model f1_score -->  44.008214124116805

In [123]:
# remove 0to21 and the last from 
Training start for  LGBMClassifier(learning_rate=0.046, n_estimators=370)
LightGBM trained successfully.. with model score -->  82.85131176837585
LightGBM trained successfully.. with model f1_score -->  43.23170091364161

In [ ]:
# remove all recfreq and keep only hh most since and 9M
Training start for  LGBMClassifier(learning_rate=0.046, n_estimators=370)
LightGBM trained successfully.. with model score -->  82.35853944822189
LightGBM trained successfully.. with model f1_score -->  41.15136336378708

In [ ]:
# remove all rec freq and keep only hh most since and 3M
Training start for  LGBMClassifier(learning_rate=0.046, n_estimators=370)
LightGBM trained successfully.. with model score -->  82.83544999841382
LightGBM trained successfully.. with model f1_score -->  42.76849305408646

In [ ]:
# remove only aarp nmas
Training start for  LGBMClassifier(learning_rate=0.046, n_estimators=370)
LightGBM trained successfully.. with model score -->  82.8640011843455
LightGBM trained successfully.. with model f1_score -->  43.36490406458603

In [ ]:
# remove 0to21 as well as aarp nmas
Training start for  LGBMClassifier(learning_rate=0.046, n_estimators=370)
LightGBM trained successfully.. with model score -->  82.8682309896687
LightGBM trained successfully.. with model f1_score -->  43.4184332762896

# remove 0to21 as well as aarp_mem_origin_dt_d which is have the corealtion with aarp_nmas
Training start for  LGBMClassifier(learning_rate=0.046, n_estimators=370)
LightGBM trained successfully.. with model score -->  82.88726511362314
LightGBM trained successfully.. with model f1_score -->  43.08774397749253


In [ ]:
# REMOVED 0TO21 VAR ONLY
Training start for  LGBMClassifier(learning_rate=0.046, n_estimators=370)
LightGBM trained successfully.. with model score -->  82.9559994501253
LightGBM trained successfully.. with model f1_score -->  43.56442577030812

In [ ]:
# changig months to years
Training start for  LGBMClassifier(learning_rate=0.046, n_estimators=370)
LightGBM trained successfully.. with model score -->  82.90312688358519
LightGBM trained successfully.. with model f1_score -->  43.36159181671688

In [ ]:
# all 62 vars
Training start for  LGBMClassifier(learning_rate=0.046, n_estimators=370)
LightGBM trained successfully.. with model score -->  82.93167806951685
LightGBM trained successfully.. with model f1_score -->  43.462117762443526


In [ ]:
# making those 2 vars as missing ind
Training start for  LGBMClassifier(learning_rate=0.046, n_estimators=370)
LightGBM trained successfully.. with model score -->  82.86611608700709
LightGBM trained successfully.. with model f1_score -->  43.27276546581242

In [ ]:
# change only pbcard
Training start for  LGBMClassifier(learning_rate=0.046, n_estimators=370)
LightGBM trained successfully.. with model score -->  82.85977137902229
LightGBM trained successfully.. with model f1_score -->  43.16819185863048

In [ ]:
# change only online purchase
Training start for  LGBMClassifier(learning_rate=0.046, n_estimators=370)
LightGBM trained successfully.. with model score -->  82.90418433491598
LightGBM trained successfully.. with model f1_score -->  43.36707885241882

In [ ]:
# without missing ind of those 2 vars
Training start for  LGBMClassifier(learning_rate=0.046, n_estimators=370)
LightGBM trained successfully.. with model score -->  82.93167806951685
LightGBM trained successfully.. with model f1_score -->  43.462117762443526

In [ ]:
# remove all 3 vars
Training start for  LGBMClassifier(learning_rate=0.046, n_estimators=370)
LightGBM trained successfully.. with model score -->  82.90947159157
LightGBM trained successfully.. with model f1_score -->  43.19555743005763

In [ ]:
# remove both vars personic and roof type
Training start for  LGBMClassifier(learning_rate=0.046, n_estimators=370)
LightGBM trained successfully.. with model score -->  82.89889707826197
LightGBM trained successfully.. with model f1_score -->  43.35948444942561

In [ ]:
# only month drop
Training start for  LGBMClassifier(learning_rate=0.046, n_estimators=370)
LightGBM trained successfully.. with model score -->  82.92533336153204
LightGBM trained successfully.. with model f1_score -->  43.27021044865264

In [ ]:
# remove roof type
Training start for  LGBMClassifier(learning_rate=0.046, n_estimators=370)
LightGBM trained successfully.. with model score -->  82.86294373301469
LightGBM trained successfully.. with model f1_score -->  43.26424870466322

In [ ]:
# remove personic cluster
Training start for  LGBMClassifier(learning_rate=0.046, n_estimators=370)
LightGBM trained successfully.. with model score -->  82.86188628168388
LightGBM trained successfully.. with model f1_score -->  43.29846412203057

In [ ]:
# all 62 vars
Training start for  LGBMClassifier(learning_rate=0.046, n_estimators=370)
LightGBM trained successfully.. with model score -->  82.93167806951685
LightGBM trained successfully.. with model f1_score -->  43.462117762443526

# save the model and load the model

# save the model

In [232]:
model13

LGBMClassifier(learning_rate=0.046, n_estimators=430)

In [ ]:
import joblib

modelname = 'models/LGBM_trained1_lr46_60v430.pkl'
#Step 1: Save the model as a pickle in a file 
joblib.dump(model13, modelname) 
  
#Step 2: Load the model from the file 
lr_from_joblib = joblib.load(modelname)  
  
#Step 3: Use the loaded model to make predictions 
# lr_from_joblib.score(X_test,y_test)

lr_from_joblib

In [234]:
lr_from_joblib.score(X_test[colseq],y_test)

0.828692884409995

In [235]:
# X_train = X_train[i].copy()
# X_test = X_test[i].copy()

In [236]:
X_train[colseq].head()

HH_NBR_CONTACTS_3M  TU_AVG_MOS_ACCTS_ON_FILE  TU_CHAZ11  AARP_MEM_X_RENEW  \
0                 4.0                     127.0      830.0               5.0   
1                 1.0                     113.0      874.0               1.0   
2                 5.0                     118.0      745.0               4.0   
3                 3.0                     148.0      683.0               5.0   
4                 5.0                     127.0      830.0               0.0   

   IND_NBR_CONTACTS_12M  IND_NBR_MOS_SINCE_LAST_MAIL  \
0                   7.0                          0.0   
1                   1.0                          0.0   
2                  15.0                          0.0   
3                   3.0                          1.0   
4                   5.0                          0.0   

   AARP_NUM_HISTPRT_FINAN_AGG_HH  IND_NBR_CONTACTS_3M  IND_NBR_CONTACTS_6M  \
0                            1.0                  4.0                  7.0   
1                            0.0                  1.0                  1.0   
2                            0.0                  5.0                  8.0   
3                            0.0                  3.0                  3.0   
4                            1.0                  5.0                  5.0   

   DS_MD014  ...  DS_MS496  HH_NBR_CONTACTS_6M  HH_NBR_CONTACTS_9M  \
0      64.0  ...  0.047712                 7.0                 7.0   
1      55.0  ...  0.047712                 1.0                 1.0   
2      74.0  ...  0.047712                 8.0                11.0   
3      70.0  ... -0.295918                 3.0                 3.0   
4      68.0  ... -0.295918                 5.0                 5.0   

   AARP_NMAS_37_REN_OT  DS_MS530  TU_BCARD_ACCTS  \
0                 77.0      20.5             5.0   
1                 69.0      18.7             2.0   
2                 57.0      21.3            14.0   
3                 44.0      26.2             5.0   
4                 33.0      30.0             5.0   

   AARP_NUM_CURPRT_OVERALL_AGG_HH  DS_MS536  AARP_ADDR_MOVE_DT_d  DS_MD002  
0                             1.0      2.61                 66.0      79.0  
1                             0.0      1.81                 53.0     101.0  
2                             0.0      2.25                 87.0     239.0  
3                             0.0      2.39                188.0      89.0  
4                             0.0      1.78                 87.0      65.0  

[5 rows x 57 columns]

In [237]:
X_test[colseq].head()

HH_NBR_CONTACTS_3M  TU_AVG_MOS_ACCTS_ON_FILE  TU_CHAZ11  AARP_MEM_X_RENEW  \
0                 0.0                     127.0      830.0               1.0   
1                 0.0                     214.0      908.0               9.0   
2                 0.0                     114.0      772.0               9.0   
3                 0.0                     118.0      791.0               3.0   
4                 0.0                      94.0      799.0               4.0   

   IND_NBR_CONTACTS_12M  IND_NBR_MOS_SINCE_LAST_MAIL  \
0                   0.0                         24.0   
1                   0.0                         24.0   
2                   2.0                         10.0   
3                   0.0                         24.0   
4                   0.0                         24.0   

   AARP_NUM_HISTPRT_FINAN_AGG_HH  IND_NBR_CONTACTS_3M  IND_NBR_CONTACTS_6M  \
0                            0.0                  0.0                  0.0   
1                            0.0                  0.0                  0.0   
2                            0.0                  0.0                  0.0   
3                            0.0                  0.0                  0.0   
4                            0.0                  0.0                  0.0   

   DS_MD014  ...  DS_MS496  HH_NBR_CONTACTS_6M  HH_NBR_CONTACTS_9M  \
0      60.0  ...  0.047712                 0.0                 2.0   
1      51.0  ...  0.047712                 0.0                 0.0   
2      79.0  ...  0.047712                 0.0                 0.0   
3      39.0  ...  0.047712                 0.0                 0.0   
4      62.0  ... -0.331806                 0.0                 0.0   

   AARP_NMAS_37_REN_OT  DS_MS530  TU_BCARD_ACCTS  \
0                 45.0      13.7             5.0   
1                 12.0      14.6             5.0   
2                 78.0      19.9             7.0   
3                 13.0       4.2             2.0   
4                 41.0      15.3             7.0   

   AARP_NUM_CURPRT_OVERALL_AGG_HH  DS_MS536  AARP_ADDR_MOVE_DT_d  DS_MD002  
0                             0.0      3.58                 87.0      63.0  
1                             0.0      2.91                 87.0     142.0  
2                             0.0      3.23                 34.0     221.0  
3                             0.0      3.23                 48.0     198.0  
4                             0.0      2.96                 88.0     139.0  

[5 rows x 57 columns]

# load the model and get deciles

In [2]:
import joblib

modelname = 'models/LGBM_trained1_lr46_60v430.pkl'

#Step 2: Load the model from the file 
lr_from_joblib = joblib.load(modelname)  
  
lr_from_joblib

LGBMClassifier(learning_rate=0.046, n_estimators=430)

In [4]:
ls = ['TU_MATCH_IND',
 'SP_EMAILABLE_IND',
 'AARP_SECONDARY_MEMBERSHIP_IND',
 'TU_AUTO_XDT_d_st',
 'TU_UNKNOWN_XDT_d_st',
 'AARP_IBX_TOT_ONLINE_DOLLARS_st',
 'IND_NBR_MOS_SINCE_LAST_MAIL',
 'IND_NBR_CONTACTS_3M',
 'HH_NBR_CONTACTS_3M',
 'IND_NBR_CONTACTS_12M',
 'IND_NBR_CONTACTS_9M',
 'IND_NBR_CONTACTS_6M',
 'HH_NBR_CONTACTS_6M',
 'HH_NBR_MOS_SINCE_LAST_MAIL',
 'HH_NBR_CONTACTS_12M',
 'HH_NBR_CONTACTS_9M',
 'TU_UNKNOWN_XDT_d',
 'AARP_PRIMARY_BIRTH_DT_d',
 'TU_AUTO_XDT_d',
 'AARP_ADDR_MOVE_DT_d',
 'AARP_PAID_THROUGH_DT_d',
 'AARP_MEM_ORIGIN_DT_d',
 'TU_BCARD_ACCTS_OPEN_0TO24M',
 'TU_BCARD_ACCTS_MOS_RCNT_OPEN',
 'TU_PUBREC_TRDLNE_DEROG_GT100DOL',
 'AARP_MEM_TERM_MO_QTY',
 'AARP_IBX_TOT_ONLINE_DOLLARS',
 'AARP_NUM_HISTPRT_OVERALL_AGG_HH',
 'AARP_IBX_NETWORTH_PR',
 'TU_ACCTS_MOS_OLD_OPEN',
 'DS_MD006',
 'DS_MD002',
 'AARP_NUM_CTCT_OVERALL_12M_AGG_HH',
 'AARP_NUM_HISTPRT_LFSTYL_AGG_HH',
 'DS_MS530',
 'DS_XB831',
 'DS_MS496',
 'AARP_LIFESTAGE_SEGMENT',
 'TU_HIGSCORE_SHFD01',
 'TU_PBCARD_ACCTS_AVG_BAL',
 'AARP_NMAS_44_CELL',
 'AARP_IBX_TRND4TELCOM_OPTCALL_SVC',
 'TU_AVG_MOS_ACCTS_ON_FILE',
 'AARP_NUM_HISTPRT_FINAN_AGG_HH',
 'DS_MS545',
 'DS_MV028',
 'AARP_IBX_HM_OWNER_RENTER_PR',
 'AARP_IBX_PERSONIC_CLUSTER',
 'DS_DF004',
 'DS_MS544',
 'TU_NUM_INS_INQ_0TO24M',
 'DS_MS410',
 'DS_MS543',
 'DS_XB977',
 'AARP_NUM_CURPRT_OVERALL_AGG_HH',
 'AARP_IBX_TOT_ONLINE_PURCHASES',
 'AARP_PRIMARY_MEM_IND',
 'AARP_IBX_HM_ROOF_TYPE',
 'RRI',
 'CRI']

---

# Calibrated

In [25]:
X_test.shape , y_test.shape

((94567, 64), (94567,))

In [26]:
len(colseq)

60

In [21]:
# method 1
from sklearn.calibration import CalibratedClassifierCV
calibrated = CalibratedClassifierCV(lr_from_joblib, cv='prefit').fit(X_test[colseq], y_test)
# y_pred = calibrated.predict(X_test)
calibrated

CalibratedClassifierCV(base_estimator=LGBMClassifier(learning_rate=0.046,
                                                     n_estimators=430),
                       cv='prefit')

In [27]:
# method 2
from sklearn.calibration import CalibratedClassifierCV
calibrated = CalibratedClassifierCV(lr_from_joblib, cv=5,method='isotonic').fit(X_test[colseq], y_test)
# y_pred = calibrated.predict(X_test)
calibrated

CalibratedClassifierCV(base_estimator=LGBMClassifier(learning_rate=0.046,
                                                     n_estimators=430),
                       cv=5, method='isotonic')

In [28]:
import joblib

modelname = 'models/LGBM_trained1_lr46_60v430_cali2.pkl'

joblib.dump(calibrated,modelname)

lr_from_joblib  = joblib.load('models/LGBM_trained1_lr46_60v430_cali2.pkl')
lr_from_joblib

CalibratedClassifierCV(base_estimator=LGBMClassifier(learning_rate=0.046,
                                                     n_estimators=430),
                       cv=5, method='isotonic')

In [29]:
X_train[colseq].head()

TU_MATCH_IND  SP_EMAILABLE_IND  AARP_SECONDARY_MEMBERSHIP_IND  \
0             0               0.0                            0.0   
1             1               0.0                            0.0   
2             1               1.0                            1.0   
3             1               0.0                            0.0   
4             0               1.0                            0.0   

   TU_AUTO_XDT_d_st  TU_UNKNOWN_XDT_d_st  AARP_IBX_TOT_ONLINE_DOLLARS_st  \
0                 1                    1                               0   
1                 0                    0                               1   
2                 0                    0                               1   
3                 1                    1                               0   
4                 1                    1                               0   

   IND_NBR_MOS_SINCE_LAST_MAIL  IND_NBR_CONTACTS_3M  HH_NBR_CONTACTS_3M  \
0                          0.0                  4.0                 4.0   
1                          0.0                  1.0                 1.0   
2                          0.0                  5.0                 5.0   
3                          1.0                  3.0                 3.0   
4                          0.0                  5.0                 5.0   

   IND_NBR_CONTACTS_12M  ...  TU_NUM_INS_INQ_0TO24M  DS_MS410  DS_MS543  \
0                   7.0  ...                    0.0 -0.057457  255332.0   
1                   1.0  ...                    1.0  0.521739  280681.0   
2                  15.0  ...                    0.0 -0.057457  320138.0   
3                   3.0  ...                    0.0 -0.057457  197187.0   
4                   5.0  ...                    0.0 -0.057457  184166.0   

   DS_XB977  AARP_NUM_CURPRT_OVERALL_AGG_HH  AARP_IBX_TOT_ONLINE_PURCHASES  \
0 -0.080860                             1.0                            1.0   
1 -0.080860                             0.0                            1.0   
2 -0.080860                             0.0                            1.0   
3 -0.080860                             0.0                            1.0   
4  0.420497                             0.0                            1.0   

   AARP_PRIMARY_MEM_IND  AARP_IBX_HM_ROOF_TYPE       RRI       CRI  
0               0.10396              -0.064294  0.359366  1.205984  
1               0.10396              -0.064294  1.344722  0.839397  
2              -0.22805               0.030804  0.476290  1.349419  
3               0.10396              -0.064294  1.548966  0.689130  
4               0.10396               0.263138  1.537114  0.698633  

[5 rows x 60 columns]

In [30]:
X_test[colseq].head()

TU_MATCH_IND  SP_EMAILABLE_IND  AARP_SECONDARY_MEMBERSHIP_IND  \
0             0               0.0                            0.0   
1             1               1.0                            0.0   
2             1               0.0                            0.0   
3             1               0.0                            0.0   
4             1               0.0                            0.0   

   TU_AUTO_XDT_d_st  TU_UNKNOWN_XDT_d_st  AARP_IBX_TOT_ONLINE_DOLLARS_st  \
0                 1                    1                               0   
1                 0                    0                               0   
2                 0                    0                               0   
3                 0                    1                               0   
4                 0                    0                               0   

   IND_NBR_MOS_SINCE_LAST_MAIL  IND_NBR_CONTACTS_3M  HH_NBR_CONTACTS_3M  \
0                         24.0                  0.0                 0.0   
1                         24.0                  0.0                 0.0   
2                         10.0                  0.0                 0.0   
3                         24.0                  0.0                 0.0   
4                         24.0                  0.0                 0.0   

   IND_NBR_CONTACTS_12M  ...  TU_NUM_INS_INQ_0TO24M  DS_MS410  DS_MS543  \
0                   0.0  ...                    0.0 -0.861484  208242.0   
1                   0.0  ...                    2.0 -0.057457  583871.0   
2                   2.0  ...                    0.0 -0.057457  583116.0   
3                   0.0  ...                    1.0 -0.057457  287765.0   
4                   0.0  ...                    0.0 -0.057457  108983.0   

   DS_XB977  AARP_NUM_CURPRT_OVERALL_AGG_HH  AARP_IBX_TOT_ONLINE_PURCHASES  \
0 -0.080860                             0.0                            1.0   
1 -0.206808                             0.0                            3.0   
2 -0.080860                             0.0                            3.0   
3 -0.080860                             0.0                            4.0   
4 -0.080860                             0.0                            1.0   

   AARP_PRIMARY_MEM_IND  AARP_IBX_HM_ROOF_TYPE       RRI       CRI  
0               0.10396              -0.064294  1.291363  0.694481  
1               0.10396               0.063493  0.982598  0.987178  
2               0.10396              -0.064294  1.154619  0.571304  
3               0.10396               0.063493  1.318983  0.855122  
4               0.10396              -0.064294  0.932342  0.848599  

[5 rows x 60 columns]

---

In [18]:
X_train.shape , X_test.shape

((219780, 65), (94207, 65))

In [19]:
from sklearn.metrics import classification_report , confusion_matrix
from collections import Counter

# train

In [20]:
# y_preds = lr_from_joblib.predict(X_train[colseq])
# type(y_preds)

# Counter(y_preds)

# print(classification_report(y_train , y_preds))

# confusion_matrix(y_train , y_preds)

# confusion_matrix(y_train , y_preds).ravel()

# # # train
# (tn, fp, fn, tp)
# array([169925,   6711,  28148,  15874])

In [21]:
# accuracy_score(y_train,y_preds)*100

# precision_score(y_train,y_preds)*100

# recall_score(y_train,y_preds)*100

# f1_score(y_train,y_preds)

# test

In [22]:
# y_preds = lr_from_joblib.predict(X_test[colseq])
# type(y_preds)

# Counter(y_preds)

# print(classification_report(y_test , y_preds))

# confusion_matrix(y_test , y_preds)

# confusion_matrix(y_test , y_preds).ravel()

# # test
# # test (tn, fp, fn, tp)
# array([72151,  3393, 12802,  6221])

In [23]:
# accuracy_score(y_test,y_preds)*100

# precision_score(y_test,y_preds)*100

# recall_score(y_test,y_preds)*100

# f1_score(y_test,y_preds)

# ROC Curve for the models trained with AUC 

In [1]:
# #define metrics
# plt.figure(figsize=(8,5))

# y_pred_proba = lr_from_joblib.predict_proba(X_test[colseq])[::,1]
# fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
# auc = metrics.roc_auc_score(y_test, y_pred_proba)

# # prod model
# y_pred_proba1 = X_test['SCORE5']
# fpr1, tpr1, _1 = metrics.roc_curve(y_test,  y_pred_proba1)
# auc1 = metrics.roc_auc_score(y_test, y_pred_proba1)

# #create ROC curve
# plt.plot(fpr,tpr,label='New Model '+" AUC="+str(round(auc,2)))
# plt.plot(fpr1,tpr1,label='Prod Model '+" AUC="+str(round(auc1,2)))
# plt.plot([0,1], [0,1], color='gray',label='No model', linestyle='--')

# plt.ylabel('True Positive Rate')
# plt.xlabel('False Positive Rate')
# plt.legend(loc=4, prop={'size': 12})
# plt.title('Train',fontsize = 20)
# plt.savefig('train_auc.jpeg')
# plt.show()

# train deciles

In [25]:
X_train[colseq].head()

TU_MATCH_IND  SP_EMAILABLE_IND  AARP_SECONDARY_MEMBERSHIP_IND  \
0             0               0.0                            0.0   
1             1               0.0                            0.0   
2             1               1.0                            1.0   
3             1               0.0                            0.0   
4             0               1.0                            0.0   

   TU_AUTO_XDT_d_st  TU_UNKNOWN_XDT_d_st  AARP_IBX_TOT_ONLINE_DOLLARS_st  \
0                 1                    1                               0   
1                 0                    0                               1   
2                 0                    0                               1   
3                 1                    1                               0   
4                 1                    1                               0   

   IND_NBR_MOS_SINCE_LAST_MAIL  IND_NBR_CONTACTS_3M  HH_NBR_CONTACTS_3M  \
0                          0.0                  4.0                 4.0   
1                          0.0                  1.0                 1.0   
2                          0.0                  5.0                 5.0   
3                          1.0                  3.0                 3.0   
4                          0.0                  5.0                 5.0   

   IND_NBR_CONTACTS_12M  ...  TU_NUM_INS_INQ_0TO24M  DS_MS410  DS_MS543  \
0                   7.0  ...                    0.0 -0.057457  255332.0   
1                   1.0  ...                    1.0  0.521739  280681.0   
2                  15.0  ...                    0.0 -0.057457  320138.0   
3                   3.0  ...                    0.0 -0.057457  197187.0   
4                   5.0  ...                    0.0 -0.057457  184166.0   

   DS_XB977  AARP_NUM_CURPRT_OVERALL_AGG_HH  AARP_IBX_TOT_ONLINE_PURCHASES  \
0 -0.080860                             1.0                            1.0   
1 -0.080860                             0.0                            1.0   
2 -0.080860                             0.0                            1.0   
3 -0.080860                             0.0                            1.0   
4  0.420497                             0.0                            1.0   

   AARP_PRIMARY_MEM_IND  AARP_IBX_HM_ROOF_TYPE       RRI       CRI  
0               0.10396              -0.064294  0.359366  1.205984  
1               0.10396              -0.064294  1.344722  0.839397  
2              -0.22805               0.030804  0.476290  1.349419  
3               0.10396              -0.064294  1.548966  0.689130  
4               0.10396               0.263138  1.537114  0.698633  

[5 rows x 60 columns]

In [27]:
X_train["Pred_prob"] = lr_from_joblib.predict_proba(X_train[colseq])[:,1]

In [26]:
X_train['decile'] = pd.qcut(X_train['Pred_prob'].rank(method='first').values, 10,
                            duplicates='drop').codes + 1

In [29]:
X_train['pred_resp'] = lr_from_joblib.predict(X_train[colseq])

In [23]:
# X_train.to_pickle('dataset/train_score.pkl')
# X_train.shape

In [27]:
X_train['actual']  = y_train #######################

In [28]:
# X_train[['SEQ_NO', 'Pred_prob', 'decile', 'actual', 'pred_resp','SCORE5','state']].to_pickle('dataset/train_graph_data11.pkl')

In [29]:
# X_train[['SEQ_NO', 'Pred_prob', 'decile', 'actual', 'pred_resp']].to_pickle('dataset/train_graph_data.pkl')

In [30]:
DF0 = X_train.groupby(['decile'], as_index=False).agg({'Pred_prob': 'count','actual': 'sum'})
DF0


DF0['actual'] =  list(DF0['actual'][::-1])

In [31]:
DF0

decile  SCORE5   actual
0       1   21978  11863.0
1       2   21978   9269.0
2       3   21978   7273.0
3       4   21978   5354.0
4       5   21978   3832.0
5       6   21978   2680.0
6       7   21978   1755.0
7       8   21978   1152.0
8       9   21978    625.0
9      10   21978    213.0

In [32]:
DF0['% mail'] = DF0['Pred_prob'].cumsum()/DF0['Pred_prob'].sum()
DF0['% mail'] = round(DF0['% mail'],4)*100

new_resp = 'cumm resp %'
DF0[new_resp] = DF0['actual'].cumsum()/DF0['actual'].sum()
DF0[new_resp] = round(DF0[new_resp],4)*100

new_cum = 'cumm RR'
DF0[new_cum]  = DF0['actual'].cumsum()/DF0['Pred_prob'].cumsum()
DF0[new_cum] = round(DF0[new_cum],4)*100

In [33]:
DF0

decile  SCORE5   actual  % mail  cumm resp %  cumm RR
0       1   21978  11863.0    10.0        26.95    53.98
1       2   21978   9269.0    20.0        48.01    48.08
2       3   21978   7273.0    30.0        64.53    43.08
3       4   21978   5354.0    40.0        76.70    38.40
4       5   21978   3832.0    50.0        85.40    34.21
5       6   21978   2680.0    60.0        91.49    30.54
6       7   21978   1755.0    70.0        95.48    27.32
7       8   21978   1152.0    80.0        98.10    24.56
8       9   21978    625.0    90.0        99.52    22.14
9      10   21978    213.0   100.0       100.00    20.03

In [51]:
X_train.drop(['Pred_prob', 'decile', 'actual'],axis=1,inplace=True)
X_train.shape

(220658, 65)

# test deciles

In [63]:
X_test["Pred_prob"] = lr_from_joblib.predict_proba(X_test[colseq])[:,1]

In [64]:
X_test['decile'] = pd.qcut(X_test['Pred_prob'].rank(method='first').values, 10,
                            duplicates='drop').codes + 1

In [65]:
X_test['pred_resp'] = lr_from_joblib.predict(X_test[colseq])

In [66]:
# X_test.to_pickle('dataset/test_score.pkl')
# X_test.shape

In [67]:
X_test['actual']  = y_test #######################

In [68]:
# X_test[['SEQ_NO', 'Pred_prob', 'decile', 'actual', 'pred_resp','SCORE5','state']].to_pickle('dataset/test_graph_data1.pkl')

In [74]:
X_test['Pred_prob'].describe(percentiles=[0.10,0.95]).reset_index()

index     Pred_prob
0  count  94567.000000
1   mean      0.199823
2    std      0.203599
3    min      0.000606
4    10%      0.009093
5    50%      0.132326
6    95%      0.622212
7    max      0.984867

In [60]:
DF0 = X_test.groupby(['decile'], as_index=False).agg({'Pred_prob': 'count','actual': 'sum'})
DF0


DF0['actual'] =  list(DF0['actual'][::-1])

In [61]:
DF0

decile  Pred_prob  actual
0       1       9457  6127.0
1       2       9457  4322.0
2       3       9456  3130.0
3       4       9457  2304.0
4       5       9457  1649.0
5       6       9456   903.0
6       7       9457   366.0
7       8       9456   121.0
8       9       9457    59.0
9      10       9457    42.0

In [62]:
DF0['% mail'] = DF0['Pred_prob'].cumsum()/DF0['Pred_prob'].sum()
DF0['% mail'] = round(DF0['% mail'],4)*100

new_resp = 'cumm resp %'
DF0[new_resp] = DF0['actual'].cumsum()/DF0['actual'].sum()
DF0[new_resp] = round(DF0[new_resp],4)*100

new_cum = 'cumm RR'
DF0[new_cum]  = DF0['actual'].cumsum()/DF0['Pred_prob'].cumsum()
DF0[new_cum] = round(DF0[new_cum],4)*100

In [63]:
DF0

decile  Pred_prob  actual  % mail  cumm resp %  cumm RR
0       1       9457  6127.0    10.0        32.21    64.79
1       2       9457  4322.0    20.0        54.93    55.24
2       3       9456  3130.0    30.0        71.38    47.86
3       4       9457  2304.0    40.0        83.49    41.99
4       5       9457  1649.0    50.0        92.16    37.08
5       6       9456   903.0    60.0        96.91    32.49
6       7       9457   366.0    70.0        98.83    28.40
7       8       9456   121.0    80.0        99.47    25.01
8       9       9457    59.0    90.0        99.78    22.30
9      10       9457    42.0   100.0       100.00    20.12

In [64]:
X_test.drop(['Pred_prob', 'decile', 'actual'],axis=1,inplace=True)
X_test.shape

(94567, 67)

# test deciles

In [63]:
X_test["Pred_prob"] = lr_from_joblib.predict_proba(X_test[colseq])[:,1]

In [34]:
X_test['decile'] = pd.qcut(X_test['SCORE5'].rank(method='first').values, 10,
                            duplicates='drop').codes + 1

In [65]:
X_test['pred_resp'] = lr_from_joblib.predict(X_test[colseq])

In [35]:
# X_test.to_pickle('dataset/test_score.pkl')
# X_test.shape

In [36]:
X_test['actual']  = y_test #######################

In [37]:
# X_test[['SEQ_NO', 'Pred_prob', 'decile', 'actual', 'pred_resp','SCORE5','state']].to_pickle('dataset/test_graph_data1.pkl')

In [38]:
DF0 = X_test.groupby(['decile'], as_index=False).agg({'SCORE5': 'count','actual': 'sum'})
DF0


DF0['actual'] =  list(DF0['actual'][::-1])

In [39]:
DF0

decile  SCORE5  actual
0       1    9421  5164.0
1       2    9421  4043.0
2       3    9420  3141.0
3       4    9421  2304.0
4       5    9421  1672.0
5       6    9420  1150.0
6       7    9421   760.0
7       8    9420   473.0
8       9    9421   232.0
9      10    9421    80.0

In [40]:
DF0['% mail'] = DF0['SCORE5'].cumsum()/DF0['SCORE5'].sum()
DF0['% mail'] = round(DF0['% mail'],4)*100

new_resp = 'cumm resp %'
DF0[new_resp] = DF0['actual'].cumsum()/DF0['actual'].sum()
DF0[new_resp] = round(DF0[new_resp],4)*100

new_cum = 'cumm RR'
DF0[new_cum]  = DF0['actual'].cumsum()/DF0['SCORE5'].cumsum()
DF0[new_cum] = round(DF0[new_cum],4)*100

In [41]:
DF0

decile  SCORE5  actual  % mail  cumm resp %  cumm RR
0       1    9421  5164.0    10.0        27.15    54.81
1       2    9421  4043.0    20.0        48.41    48.86
2       3    9420  3141.0    30.0        64.92    43.69
3       4    9421  2304.0    40.0        77.04    38.88
4       5    9421  1672.0    50.0        85.83    34.66
5       6    9420  1150.0    60.0        91.88    30.91
6       7    9421   760.0    70.0        95.87    27.65
7       8    9420   473.0    80.0        98.36    24.82
8       9    9421   232.0    90.0        99.58    22.34
9      10    9421    80.0   100.0       100.00    20.19

# hyperparameter tuning with Optuna

In [123]:
X_train.shape , X_test.shape

((220658, 318), (94567, 318))

In [124]:
X_train = X_train[colseq]
X_test = X_test[colseq]
X_train.shape , X_test.shape

((220658, 57), (94567, 57))

In [125]:
import optuna

In [126]:
# !pip install verstack

In [127]:
from verstack import LGBMTuner

In [132]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, accuracy_score
import optuna

def objective(trial):
    params = {
        "objective": "regression",
        "metric": "accuracy",
#         "n_estimators": 400,
        "verbosity": -1,
        "bagging_freq": 1,
#         "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "learning_rate": trial.suggest_float("learning_rate",1e-4,0.3,log=True),
#         "num_leaves": trial.suggest_int("num_leaves", 2, 2**10),
#         "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
#         "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
                'n_estimators': trial.suggest_int('n_estimators',100,700,30),

    }

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train, verbose=False)
    predictions = model.predict(X_test)
#     rmse = mean_squared_error(y_val, predictions, squared=False)
    acc = accuracy_score(y_test, predictions)
    return acc

In [129]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

[I 2023-10-09 06:35:39,884] A new study created in memory with name: no-name-f61f5bb5-0557-48eb-ae49-0dd17deb775e


[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-10-09 06:36:33,232] Trial 0 finished with value: 0.8179068808358095 and parameters: {'learning_rate': 0.011955626615769245, 'n_estimators': 280}. Best is trial 0 with value: 0.8179068808358095.


[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-10-09 06:36:37,537] Trial 1 finished with value: 0.7988410333414404 and parameters: {'learning_rate': 0.0004851150951899107, 'n_estimators': 250}. Best is trial 0 with value: 0.8179068808358095.


...

[I 2023-10-09 06:39:46,156] Trial 27 finished with value: 0.8290524178624679 and parameters: {'learning_rate': 0.06451979163312943, 'n_estimators': 490}. Best is trial 21 with value: 0.8294542493681728.


[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-10-09 06:39:55,016] Trial 28 finished with value: 0.8280372645848975 and parameters: {'learning_rate': 0.026677547035970597, 'n_estimators': 610}. Best is trial 21 with value: 0.8294542493681728.


[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-10-09 06:40:00,233] Trial 29 finished with value: 0.8165004705658422 and parameters: {'learning_rate': 0.011801356869900935, 'n_estimators': 250}. Best is trial 21 with value: 0.8294542493681728.


In [131]:
print('Best hyperparameters:', study.best_params)
print('Best RMSE:', study.best_value)

Best hyperparameters: {'learning_rate': 0.09980062678657535, 'n_estimators': 700}
Best RMSE: 0.8294542493681728


In [ ]:
# optuna without HIG score
learning_rate= 0.09621200251712454, 
n_estimators= 610

In [ ]:
# optuna without HIG score
learning_rate =0.09980062678657535,
n_estimators= 700

In [ ]:
# optuna 1
Best hyperparameters: {'learning_rate': 0.028098656983592478, 'num_leaves': 676, 'subsample': 0.9993452736301868, 'colsample_bytree': 0.5064801186373241, 'min_data_in_leaf': 66}
Best RMSE: 0.8305857222921315

In [ ]:
# optuna 2
learning_rate 0.028129178131793758,
num_leaves 650,
subsample 0.9210055040474047,
colsample_bytree 0.44048022891844446,
min_data_in_leaf 53,
n_estimators 450

In [ ]:
# optuna 3
Best hyperparameters: {'learning_rate': 0.050979337220196846, 'n_estimators': 900}

In [ ]:
# optuna 4
Best hyperparameters: {'learning_rate': 0.0843256322346568, 'n_estimators': 300}

In [ ]:
# optuna 5
learning_rate= 0.09624443449629956,
colsample_bytree= 0.45103435121611407,
n_estimators= 900

In [ ]:
# optuna 6 with 42 vars only which we derived from drill vars
learning_rate= 0.12145090367766112,
n_estimators= 520

In [ ]:
# optuna model withou HIG Score
learning_rate =0.07997706392625581,
n_estimators= 430

# Randomized search CV

In [14]:
X_train.shape , X_test.shape

((220658, 62), (94567, 62))

In [15]:
X_train.head()

TU_MATCH_IND  SP_EMAILABLE_IND  AARP_SECONDARY_MEMBERSHIP_IND  \
0             0               0.0                            0.0   
1             1               0.0                            0.0   
2             1               1.0                            1.0   
3             1               0.0                            0.0   
4             0               1.0                            0.0   

   TU_AUTO_XDT_d_st  TU_UNKNOWN_XDT_d_st  AARP_IBX_TOT_ONLINE_DOLLARS_st  \
0                 1                    1                               0   
1                 0                    0                               1   
2                 0                    0                               1   
3                 1                    1                               0   
4                 1                    1                               0   

   IND_NBR_MOS_SINCE_LAST_MAIL  IND_NBR_CONTACTS_3M  HH_NBR_CONTACTS_3M  \
0                            0                    4                   4   
1                            0                    1                   1   
2                            0                    5                   5   
3                            1                    3                   3   
4                            0                    5                   5   

   IND_NBR_CONTACTS_12M  ...  DS_MS543  DS_XB977  TU_NUM_INS_INQ_0TO21M  \
0                     7  ...  255332.0 -0.080860                    0.0   
1                     1  ...  280681.0 -0.080860                    1.0   
2                    15  ...  320138.0 -0.080860                    0.0   
3                     3  ...  197187.0 -0.080860                    0.0   
4                     5  ...  184166.0  0.420497                    0.0   

   AARP_NUM_CURPRT_OVERALL_AGG_HH  AARP_IBX_TOT_ONLINE_PURCHASES  \
0                             1.0                            1.0   
1                             0.0                            1.0   
2                             0.0                            1.0   
3                             0.0                            1.0   
4                             0.0                            1.0   

   AARP_PRIMARY_MEM_IND  AARP_IBX_HM_ROOF_TYPE  DS_XB875       RRI       CRI  
0               0.10396              -0.064294   16980.0  0.359366  1.205984  
1               0.10396              -0.064294   33100.0  1.344722  0.839397  
2              -0.22805               0.030804   38060.0  0.476290  1.349419  
3               0.10396              -0.064294   21020.0  1.548966  0.689130  
4               0.10396               0.263138   38240.0  1.537114  0.698633  

[5 rows x 62 columns]

In [16]:
X_test.head()

TU_MATCH_IND  SP_EMAILABLE_IND  AARP_SECONDARY_MEMBERSHIP_IND  \
0             0               0.0                            0.0   
1             1               1.0                            0.0   
2             1               0.0                            0.0   
3             1               0.0                            0.0   
4             1               0.0                            0.0   

   TU_AUTO_XDT_d_st  TU_UNKNOWN_XDT_d_st  AARP_IBX_TOT_ONLINE_DOLLARS_st  \
0                 1                    1                               0   
1                 0                    0                               0   
2                 0                    0                               0   
3                 0                    1                               0   
4                 0                    0                               0   

   IND_NBR_MOS_SINCE_LAST_MAIL  IND_NBR_CONTACTS_3M  HH_NBR_CONTACTS_3M  \
0                           24                    0                   0   
1                           24                    0                   0   
2                           10                    0                   0   
3                           24                    0                   0   
4                           24                    0                   0   

   IND_NBR_CONTACTS_12M  ...  DS_MS543  DS_XB977  TU_NUM_INS_INQ_0TO21M  \
0                     0  ...  208242.0 -0.080860                    0.0   
1                     0  ...  583871.0 -0.206808                    2.0   
2                     2  ...  583116.0 -0.080860                    0.0   
3                     0  ...  287765.0 -0.080860                    1.0   
4                     0  ...  108983.0 -0.080860                    0.0   

   AARP_NUM_CURPRT_OVERALL_AGG_HH  AARP_IBX_TOT_ONLINE_PURCHASES  \
0                             0.0                            1.0   
1                             0.0                            3.0   
2                             0.0                            3.0   
3                             0.0                            4.0   
4                             0.0                            1.0   

   AARP_PRIMARY_MEM_IND  AARP_IBX_HM_ROOF_TYPE  DS_XB875       RRI       CRI  
0               0.10396              -0.064294   31080.0  1.291363  0.694481  
1               0.10396               0.063493   40060.0  0.982598  0.987178  
2               0.10396              -0.064294   35620.0  1.154619  0.571304  
3               0.10396               0.063493   28140.0  1.318983  0.855122  
4               0.10396              -0.064294   13660.0  0.932342  0.848599  

[5 rows x 62 columns]

In [64]:
LGBMClassifier()

LGBMClassifier()

In [17]:
 np.arange(0,751,50)

array([  0,  50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600,
       650, 700, 750])

In [21]:
# rs_params = {
#     'n_estimators': np.arange(100,500,20),
#         'bagging_fraction': np.arange(0.0, 1.0,0.2),
#         'bagging_frequency': np.arange(0, 9,1),
#         'feature_fraction': np.arange(0.0, 0.9,0.2),
#         'max_depth': np.arange(0,31,3),
#         'min_data_in_leaf': np.arange(100, 200,10),
#         'num_leaves': np.arange(50,500,20)
# }
rs_params = {
    'num_leaves': np.arange(0,751,50),
        'max_depth': np.arange(0 ,1001,50 )
}

# Initialize a RandomizedSearchCV object using 5-fold CV-
rs_cv = GridSearchCV(estimator=LGBMClassifier(learning_rate= 0.054999999999999966,n_estimators=460),
                           param_grid=rs_params,
                           cv = 3,
                           n_jobs=-1,
                           scoring='accuracy',
#                            n_iter=150,
                     verbose=1)

# Train on training data-
rs_cv.fit(X_train, y_train,verbose=1)

Fitting 3 folds for each of 336 candidates, totalling 1008 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 96 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 258 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done 608 tasks      | elapsed: 12.0min
[Parallel(n_jobs=-1)]: Done 1008 out of 1008 | elapsed: 19.7min finished


GridSearchCV(cv=3,
             estimator=LGBMClassifier(learning_rate=0.054999999999999966,
                                      n_estimators=460),
             n_jobs=-1,
             param_grid={'max_depth': array([   0,   50,  100,  150,  200,  250,  300,  350,  400,  450,  500,
        550,  600,  650,  700,  750,  800,  850,  900,  950, 1000]),
                         'num_leaves': array([  0,  50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600,
       650, 700, 750])},
             scoring='accuracy', verbose=1)

In [66]:
import pickle

model_pkl_file = "LGBM_trained10.pkl"  

with open(model_pkl_file, 'wb') as file:  
    pickle.dump(rs_cv, file)

# load model from pickle file
with open(model_pkl_file, 'rb') as file:  
    model = pickle.load(file)
model

GridSearchCV(cv=3, estimator=LGBMClassifier(), n_jobs=-1,
             param_grid={'colsample_bytree': [0.2, 0, 4, 0.6, 0.8, 1.0],
                         'learning_rate': array([0.1  , 0.095, 0.09 , 0.085, 0.08 , 0.075, 0.07 , 0.065, 0.06 ,
       0.055, 0.05 , 0.045, 0.04 , 0.035, 0.03 , 0.025, 0.02 , 0.015,
       0.01 , 0.005]),
                         'n_estimators': array([100, 120, 140, 160, 180, 200, 220, 240, 260, 280, 300, 320, 340,
       360, 380, 400, 420, 440, 460, 480])},
             scoring='accuracy', verbose=1)

In [22]:
rs_cv.best_params_

{'max_depth': 0, 'num_leaves': 50}

In [20]:
rs_cv.best_estimator_.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': 0,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'num_leaves': 100,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': 'warn',
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [ ]:
# ls = list(np.arange(8,129,4))

# ls1 = []
# for i in ls:
#     ls1.append((i,i,i))

# len(ls1)

In [ ]:
# mlp_gs = MLPClassifier(max_iter=200,early_stopping=True)
# parameter_space = {
#     'hidden_layer_sizes': [(8,8,8),(64,64,64),(80,80,80),(100,100,100),(128,128,128),(128,64,8),(128,100,64,8)
#                            ,(64,64,8),(64,8,8),(128,128,64)],
#     'activation': ['tanh', 'relu'],
#     'solver': ['sgd', 'adam'],
#     'alpha': [0.0001, 0.05],
#     'learning_rate': ['constant','adaptive'],
# }
# from sklearn.model_selection import GridSearchCV
# model12 = GridSearchCV(mlp_gs, parameter_space, n_jobs=-1, cv=5)
# model12.fit(X_train, y_train) # X is train samples and y is the corresponding labels
# print('Best parameters found:\n', model12.best_params_)

In [ ]:
# import pickle

# model_pkl_file = "tuned_NN.pkl"  

# with open(model_pkl_file, 'wb') as file:  
#     pickle.dump(model12, file)

# # load model from pickle file
# with open(model_pkl_file, 'rb') as file:  
#     model = pickle.load(file)
# model